In [ ]:
!pip install emoji

In [ ]:
!pip install -U deep-translator

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from deep_translator import GoogleTranslator
import emoji

In [ ]:
train = pd.read_csv('./datasets/default/data.csv')
# print(data.shape)

### Original (Train + Test) Data

test merupakan 1000 data terakhir yang labelnya kosong

In [ ]:
train

In [ ]:
print(train.isnull().sum())

Terdapat 2 data null pada tweet data train, drop dua data tersebut

In [ ]:
train.dropna(subset=['tweet'], inplace=True)
print(train.isnull().sum())

In [ ]:
# train.to_csv('./datasets/no_null/train.csv', index=False)

In [ ]:
train.describe()

## Preprocessing

### Count Changes Function

Fungsi ini digunakan untuk mengetahui berapa banyak perubahan yang dilakukan untuk tiap proses

In [ ]:
def count_changes(initial_df, df_after_step):
    return (initial_df != df_after_step).any(axis=1).sum()

### String Parsing

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].astype(str)
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

In [ ]:
train.info()

### Hilangkan Mention dan Link

Menghilangkan [USERNAME] dan [URL] 

In [ ]:
def remove_mention_link(text):
    result = re.sub(r"\[USERNAME\]", "", text)
    result = re.sub(r"\[URL\]", "", result)
    result = re.sub(r" {2,}", " ", result)
    return result.strip()

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].apply(lambda x : remove_mention_link(x))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

In [ ]:
train

### Hilangkan Hastag

In [ ]:
def split_camel_case_hashtag(text):
    words = text.split()
    for i, word in enumerate(words):
        if word.startswith('#'):
            # Remove the leading '#' and split camel case words
            words[i] = words[i].replace('#', '')
            words[i] = re.sub(r'([a-z])([A-Z])', r'\1 \2', words[i])
    return ' '.join(words).lower()

In [ ]:
# train['has_hashtag'] = train['tweet'].str.contains('#', case=False)

# # Count the number of rows where 'has_hashtag' is True
# num_hashtags = train['has_hashtag'].sum()
# num_hashtags

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].apply(lambda x : split_camel_case_hashtag(x))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

In [ ]:
train

### Lowercasing

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].str.lower()
changes = count_changes(initial_df, train)
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

In [ ]:
train

### Hilangkan String Emoticon

String Emoticon merupakan emoticon wajah menggunakan string = :) , :( , ;| dan lainnya

In [ ]:
def remove_string_emoticons(text):
    # Remove emoticons (e.g., :), :-), :D, ...)
    emoticon_pattern = r'(?::|;|=)(?:-)?(?:\)|\(|D|P)'
    text = re.sub(emoticon_pattern, '', text)
    
    return text

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].apply(lambda x : remove_string_emoticons(x))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

In [ ]:
# train

### Remove Emoji

In [ ]:
def remove_emoji(text):
    # Mengubah emoji menjadi string
    # 😆 -> :)
    text = emoji.demojize(text)
    
    # Menghapus string emoji
    text = re.sub(r':[a-z_]+:', '', text)
    return text

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].apply(lambda x : remove_emoji(x))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

In [ ]:
# train

### Remove Tanda Baca

Semua tanda baca kecuali tanda petik satu di remove menggunakan spasi, sedangkan tanda petik satu di remove tanpa spasi

In [ ]:
punctuations = string.punctuation
punctuations

In [ ]:
def clean_punctuations(text):
    translator = str.maketrans(punctuations.replace("'", ''), ' ' * (len(punctuations) - 1))
    text = text.translate(translator)
    text = re.sub(' +', ' ', text)
    text = re.sub(r'\'', '', text)
    return text

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].apply(lambda x : clean_punctuations(x))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

In [ ]:
# train

### Translate Inggris ke Indonesia

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
translator = GoogleTranslator(source='en', target='id')

In [ ]:
def translate_word(input_text):
    # result = text
    # for word in text.split():
    #     if wordnet.synsets(word):
    #         result = result.replace(word, translator.translate(text=word).lower())
    result = translator.translate(text=input_text).lower()
    return result

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].apply(lambda x: translate_word(x))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

### Hilangkan Karater Berulang

In [ ]:
def clean_repeating_characters(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].apply(lambda x: clean_repeating_characters(x))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

In [ ]:
# train

### Remove HTML Tags

In [ ]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].apply(lambda x : remove_html(x))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

### Hilangkan URL

In [ ]:
def clean_URLs(text):
    return re.sub(r"((www.[^s]+)|(http\S+))","",text)

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].apply(lambda x : clean_URLs(x))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

### Hilangkan Numeric Data

In [ ]:
def clean_numeric(text):
    return re.sub('[0-9]+', '', text)

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].apply(lambda x: clean_numeric(x))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

In [ ]:
# train

### Remove Kata yang hanya Satu Huruf

In [ ]:
def clean_single_alphabet_words(text):
    # Define a regular expression pattern to match single alphabet words
    pattern = r'\b[a-zA-Z]\b'
    
    # Use re.sub to replace matched patterns with an empty string
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text.strip()

In [ ]:
initial_df = train.copy()
train['tweet'] = train['tweet'].apply(lambda x: clean_single_alphabet_words(x))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

### Remove Slangword

Slang words dari https://github.com/jakartaresearch/maleo/blob/master/maleo/preprocessing/slang_dict.json

In [ ]:
slangs_1 = {"1pun": "satupun", "1st": "pertama", "abal2": "abal-abal", "abis": "habis", "about": "tentang", "abu2": "abu-abu", "acah2": "acah-acah", "acara2": "acara-acara", "acc": "terima", "account": "akun", "acr": "acara", "activists": "aktifis", "actually": "sebenarnya", "ad": "ada", "ada2": "ada-ada", "adl": "adalah", "adlh": "adalah", "administ": "administrasi", "adoption": "adopsi", "advertisement": "iklan", "advocate": "menganjurkan", "aeard": "penghargaan", "agama2": "agama-agama", "agma": "agama", "agree": "setuju", "ahoax": "ahok", "aj": "saja", "aja": "saja", "ajah": "saja", "ajak2": "ajak-ajak", "aje": "saja", "ak": "aku", "akar2nya": "akar-akarnya", "akaun": "akun", "akherat": "akhirat", "akhir2": "akhir-akhir", "akn": "akan", "aksi2": "aksi-aksi", "aktifis2": "aktivis-aktivis", "aktivi": "aktivis", "aktivis2": "aktivis-aktivis", "akun2": "akun-akun", "ala": "menurut", "alang-alang": "untuk", "alasn": "alas an", "aleg": "caleg", "alih2": "alih-alih", "all": "semua", "alsn": "alasan", "alesan": "alasan", "ama": "sama", "america": "amerika", "american": "amerika", "ampe": "sampai", "amrik": "amerika", "anak2": "anak-anak", "anak2nya": "anak-anaknya", "anak2x": "anak-anak", "ancur": "hancur", "and": "dan", "anda": "kamu", "ane": "saya", "aneh2": "aneh-aneh", "ang": "anggota", "angg": "anggota", "anggep": "anggap", "anggota2": "anggota-anggota", "ank": "anak", "annoying": "menyebalkan", "another": "lain", "antek2nya": "antek-anteknya", "anti2": "anti-anti", "antum": "kalian", "apa2": "apa-apa", "apalg": "apalagi", "ape": "apa", "apkh": "apakah", "aq": "aku", "are": "adalah", "art": "seni", "article": "artikel", "artis2": "artis-artis", "asekk": "asik", "asep": "asap", "aset2": "aset-aset", "association": "asosiasi", "assume": "asumsi", "at": "di", "ati2": "hati-hati", "ato": "atau", "ats": "atas", "atur2": "atur-atur", "atvsi": "asosiasi televisi swasta indonesia", "atw": "atau", "au": "angkatan udara", "awak": "kamu", "ayat2": "ayat-ayat", "azaab": "azab", "b2": "berdua", "baca2": "baca-baca", "back": "kembali", "backup": "cadangan", "bad": "buruk", "bae": "baik", "bagamana": "bagaimana", "bagian2": "bagian-bagian", "bahawa": "bahwa", "bakal": "akan", "bala2": "bakwan", "baper": "bawa perasaan", "bar2": "bar-bar", "bareng2": "bareng-bareng", "baru2": "baru-baru", "based": "berdasarkan", "bash": "pesta", "bawa2": "bawa-bawa", "bbrp": "beberapa", "bdaya": "budaya", "bdk": "budak", "bdk2": "budak-budak", "beauty": "cantik", "because": "karena", "been": "telah", "begimana": "bagaimana", "bego": "bodoh", "being": "menjadi", "bejimana": "bagaimana", "bela2": "bela-bela", "belom": "belum", "bem": "badan eksekutif mahasiswa", "bencong2": "banci-banci", "bener": "benar", "bener2": "benar-benar", "benih2": "benih-benih", "bentuk2nya": "bentuk-bentuknya", "beramai2": "beramai-ramai", "berbusa2": "berbusa-busa", "berhenti2": "berhenti berhenti", "berita2": "berita-berita", "berkoar2": "berkoar-koar", "berlarut2": "berlarut-larut", "berlomba2": "berlomba-lomba", "berpilaku": "berperilaku", "bersama2": "bersama-sama", "bersatu1": "bersatu satu", "bersiap2": "bersiap-siap", "bersiap2lah": "bersiap-siaplah", "bersih2": "bersih-bersih", "bertahun2": "bertahun-tahun", "berturut2": "berturut-turut", "berubah2": "berubah-ubah", "besar2an": "besar-besaran", "besok2": "besok-besok", "best": "terbaik", "bet": "banget", "better": "lebih baik", "betul": "benar", "betul2": "betul-betul", "bg": "bang", "bgm": "bagaimana", "bgmn": "bagaimana", "bgs": "bagus", "bgst": "bangsat", "bgt": "banget", "bgtu": "begitu", "bhai": "bye", "bhkan": "bahkan", "bhubung": "berhubung", "bhw": "bahwa", "bhwa": "bahwa", "bhy": "bahaya", "biasa2": "biasa-biasa", "bibit2": "bibit-bibit", "bible": "injil", "big": "besar", "bigot": "fanatik", "bikin": "membuat", "bin": "badan intelligent negara", "biot2": "fanatik-fanatik", "bisa2": "bisa-bisa", "bkannya": "bukannya", "bkl": "akan", "bkmt": "badan kontak majelis taklim", "bkn": "bikin", "bknnya": "bukannya", "blak2an": "terus terang", "blanche": "putih", "blas": "balas", "blg": "bilang", "blh": "boleh", "blhkan": "bolehkan", "blm": "belum", "bln": "bulan", "blng": "bilang", "bls": "balas", "bnda": "bunda", "bntk": "bentuk", "bntr": "bentar", "bnyk": "banyak", "bnyk2": "banyak-banyak", "bodoh2": "bodoh-bodoh", "bole": "boleh", "bopi": "badan olahraga profesional indonesia", "boro-boro": "jangankan", "bpk": "bapak", "bpk2": "bapak-bapak", "brader": "adik", "brhadapan": "berhadapan", "brjuanglh": "berjuanglah", "bro": "kakak", "broter": "kakak", "broters": "kakak-kakak", "brp": "berapa", "brsdia": "bersedia", "brstu": "bersatu", "bs": "bisa", "bsa": "bisa", "bsk": "besok", "bsr": "besar", "btl": "batal", "btol": "botol", "btw": "ngomong-ngomong", "buah2an": "buah-buahan", "buang2": "buang-buang", "buat2": "buat-buat", "budak2": "budak-budak", "budha": "buddha", "bukn": "bukan", "bukti2": "bukti-bukti", "buku2": "buku-buku", "bule2": "bule-bule", "bunuh2an": "bunuh-bunuhan", "but": "tapi", "bw": "bawa", "bw2": "bawa-bawa", "bwt": "buat", "bye": "selamat tinggal", "byk": "banyak", "cabe": "cabai", "cakap": "berbicara", "calon2": "calon-calon", "cantik2": "cantik-cantik", "capres": "calon presiden", "capresss": "calon presiden", "car": "mobil", "cause": "karena", "cawapres": "calon wakil presiden", "cebong2": "cebong-cebong", "cepat2": "cepat cepat", "cew": "cewek", "cewe": "cewek", "children": "anak anak", "choice": "pilihan", "chrismas": "natal", "christmas": "natal", "cibi2an": "cabe-cabean", "cikgu": "guru", "cine": "bioskop", "cinema": "bioskop", "ciri2": "ciri-ciri", "cita2ku": "cita-cita ku", "cjr": "coboy junior", "ckp": "cukup", "club": "kelompok", "cm": "cuma", "cnth": "contoh", "come": "datang", "community": "komunitas", "completely": "dengan lengkap", "con": "dengan", "confess": "mengaku", "cos": "karena", "couple": "pasangan", "couples": "pasangan-pasangan", "cow": "cowok", "cowo2": "cowo-cowo", "create": "membuat", "crny": "caranya", "cth": "contoh", "cuap": "berkata", "cuap2": "berkata-kata", "curate": "pembantu pendeta", "curious": "ingin tau", "cust": "customer", "cwe": "cewek", "cwo": "cowok", "d": "ada", "dah": "deh", "dak2": "dak-dak", "dalih2": "dalih-dalih", "dalil2": "dalil-dalil", "damn": "mengutuk", "danger": "bahaya", "dangerous": "bahaya", "dap": "partai aksi demokratis", "dapil": "daerah pemilihan", "dbenarkn": "dibenarkan", "dead": "mati", "death": "mati", "debate": "debat", "dedengkot2": "tokoh-tokoh", "dek": "adik", "denger": "dengar", "denger2": "dengar-dengar", "dg": "dengan", "dgjantan": "dengan jantan", "dgn": "dengan", "dgnnya": "dengannya", "dgr": "dengar", "dh": "dah", "diam2": "diam-diam", "dibawa2": "dibawa-bawa", "dibuthkan": "dibutuhkan", "die": "dia", "diem2": "diam diam", "dijauhkn": "dijauhkan", "dik": "adik", "dikejar2": "dikejar-kejar", "dikiitt": "sedikit", "dikit2": "dikit-dikit", "dikti": "direktoral jendral pendidikan tinggi", "dimana2": "dimana-mana", "diobok2": "diaduk-aduk", "dipake": "dipakai", "direhap": "direhab", "diselametin": "diselamatkan", "disembuhkn": "disembuhkan", "ditakut2": "ditakut-takut", "diy": "daerah ibukota yogyakarta", "dki": "daerah khusus ibukota", "dkk": "dan kawan kawan", "dkt": "dekat", "dkwan": "dakwaan", "dll": "dan lain lain", "dlm": "dalam", "dlu": "dulu", "dmaksud": "dimaksud", "dn": "dan", "dnytakn": "dinyatakan", "doang": "saja", "dogma": "ajaran", "dogma2": "ajaran-ajaran", "dongok": "bodoh", "donk": "dong", "dpn": "depan", "dpt": "dapat", "dr": "dokter", "dri": "dari", "drpd": "daripada", "dsana": "disana", "dsar": "dasar", "dsb": "dan sebagainya", "dsr": "dasar", "dst": "dan seterusnya", "dtg": "datang", "dtng": "datang", "dua2": "dua-dua", "dua2nya": "dua-duanya", "duit": "uang", "dulu": "dahulu", "dy": "dia", "easter": "paskah", "ejek2": "ejek-ejek", "elo": "lo", "elu": "lu", "elu2": "anda-anda", "emang": "memang", "emg": "memang", "emng": "memang", "engga": "enggak", "enggak": "tidak", "entar": "nanti", "ente": "kamu", "eps": "episode", "equality": "persamaan", "etc": "dan lain-lain", "fahami": "pahami", "fair": "adil", "fak": "fakultas", "fakta2": "fakta-fakta", "fanatik2": "fanatik", "fans": "fan", "fav": "favorit", "fb": "facebook", "fesbuk": "facebook", "filem": "film", "filem2": "film-film", "fire": "api", "for": "untuk", "foto2": "foto-foto", "friday": "Jumat", "from": "dari", "funny": "menyenangkan", "g": "enggak", "ga": "enggak", "gada": "enggak ada", "gaduh2": "gaduh-gaduh", "gak": "enggak", "galak2": "galak-galak", "gamau": "enggak mau", "gambar2": "gambar-gambar", "gampang": "mudah", "ganteng2": "ganteng-ganteng", "gapapa": "tidak apa-apa", "gara2": "gara-gara", "garis2": "garis-garis", "gatau": "enggak tau", "gays": "teman-teman", "gblk": "bodoh", "gd": "gusdur", "geboy2": "geboy-geboy", "gede": "besar", "gedek": "kesal", "gegara": "gara-gara", "gelak2": "menghisap-menghisap", "gembala2": "gembala-gembala", "gender": "jenis kelamin", "gene": "begini", "geng2": "kelompok-kelompok", "gereja2": "gereja-gereja", "get": "memperoleh", "ggl": "gagal", "gib": "gerakan indonesia beradab", "gile": "gila", "gimana": "bagaimana", "ginger": "jahe", "gitu": "begitu", "gitu2": "begitu-gitu", "gjd": "tidak jadi", "gk": "enggak", "glongn": "golongan", "gmn": "bagaimana", "gmna": "bagaimana", "goblok": "bodoh", "golongan2": "golongan-golongan", "gpp": "enggak apa-apa", "grepe2": "memegang-megang", "grja": "gereja", "group": "grup", "grup2": "grup-grup", "gt": "begitu", "gua": "saya", "gub": "gubernur", "gue": "saya", "guna": "menggunakan", "guru2": "guru-guru", "gusto": "suka", "guys": "teman-teman", "gw": "gue", "habis2": "habis-habis", "habis2an": "habis-habisan", "hadist2": "hadist-hadist", "hak2": "hak hak", "hakim2": "hakim-hakim", "hal2": "hal-hal", "halalkn": "halalkan", "hamba2": "hamba-hamba", "hantu2": "hantu-hantu", "hapuskn": "hapuskan", "hari2": "hari-hari", "harom": "haram", "hasil2": "hasil-hasil", "hate": "benci", "hati2": "hati-hati", "hdp": "hidup", "here": "disini", "hey2": "hai-hai", "hindi": "hindu", "hit": "memukul", "hny": "hanya", "hnya": "hanya", "holiday": "hari libur", "homophobic": "homofobia", "host": "pembawa acara", "hot2nya": "hot-hotnya", "how": "bagaimana", "hp": "handphone", "hr": "hari", "hrm": "haram", "hrs": "harus", "hrsnya": "harusnya", "hsl": "hasil", "humanity": "kemanusiaan", "i": "anda", "ibdh": "ibadah", "ibu2": "ibu-ibu", "icon": "ikon", "icons": "ikon", "idap": "menderita", "idc": "saya tidak peduli", "ide2": "ide-ide", "idk": "saya tidak tahu", "idup": "hidup", "iklan2": "iklan-iklan", "ikut2an": "ikut-ikutan", "ilah": "Allah", "illah": "Allah", "illahi": "Allah", "image": "gambar", "indinesia": "indonesia", "indo": "indonesia", "indon": "Indonesia", "injak2": "injak injak", "interview": "wawancara", "isu2": "isu-isu", "isu2nya": "isu-isunya", "isue": "isu", "itj": "Indonesia tanpa jil", "ito": "itu", "itu2": "itu-itu", "jad2an": "jadi-jadian", "jalan2": "jalan-jalan", "jan": "jangan", "jangan2": "jangan-jangan", "jargon2": "jargon jargon", "jasa2": "jasa-jasa", "jatohin": "menjatuhkan", "jauh2": "jauh-jauh", "jawapan": "jawaban", "jd": "jadi", "jdi": "jadi", "je": "saja", "jelas2": "jelas-jelas", "jg": "juga", "jgk": "juga", "jgn": "jangan", "ji2k": "jijik", "jiwa2": "jiwa-jiwa", "jjk": "jijik", "jk": "jika", "jkowi": "jokowi", "jkt": "jakarta", "jkw": "jokowi", "jlas": "jelas", "jln": "jalan", "jmn": "jaman", "jng": "jangan", "joget2": "joget-joget", "joke": "lelucon", "jomblo2": "jomblo-jomblo", "jual2": "jual-jual", "jugak": "juga", "just": "hanya", "jwab": "jawab", "jwb": "jawab", "k": "kak", "kadang2": "kadang kadang", "kader2": "kader-kader", "kafir2": "kafir-kafir", "kaga": "kagak", "kagak": "tidak", "kailangan": "dibuthkan", "kalo": "kalau", "kaltim": "kalimantan timur", "kalsel": "kalimantan selatan", "kalbar": "kalimantan barat", "kamu2": "kamu-kamu", "kan": "akan", "kanwil": "kantor wilayah", "kapolri": "Kepala Polisi Republik Indonesia", "karep": "terserah", "karep2": "terserah-terserah", "kasus2": "kasus-kasus", "kat": "kata", "kata2": "kata-kata", "kata2mu": "kata-kata kamu", "kau": "anda", "kawan": "teman", "kawan2": "teman-teman", "kayak": "seperti", "kayaknya": "sepertinya", "kbaikan": "kebaikan", "kbthn": "kebutuhan", "kdg": "kadang", "kdg2": "kadang-kadang", "kdng": "kadang", "kearab2an": "kearab-araban", "kebalik2": "kebalik-balik", "kebanggan": "kebanggaan", "kebiadaban2": "kebiadaban-kebiadaban", "kebranian": "keberanian", "kec": "kecamatan", "kedah": "harus", "kedlm": "kedalam", "kejang2": "kejang-kejang", "kejar2": "kejar-kejar", "kek": "kayak", "keknya": "kayaknya", "kel": "keluarga", "kelonggaran2": "kelonggaran-kelonggaran", "kemarin2": "kemarin-kemarin", "kemrin": "kemarin", "kena": "sentuh", "kercep": "kerja cepat", "keren2": "keren-keren", "kesewenang2": "kesewenang-wenangan", "kesihatan": "kesehatan", "kespro": "konsep dasar kesehatan reproduksi", "kessel": "kesal", "ketakutan2": "ketakutan-ketakutan", "ketawa2": "ketawa-ketawa", "ketum": "ketua umum", "kezel": "kesal", "kira2": "kira-kira", "kisah2": "kisah-kisah", "kl": "kalo", "klau": "kalau", "klepek2": "klepek-klepek", "klean": "kalian", "klmpk": "kelompok", "klo": "kalo", "kls": "kelas", "klu": "kalau", "kluarga": "keluarga", "klw": "kalau", "km": "kami", "kmaksiatan": "kemaksiatan", "kmna": "kemana", "kmrn": "kemarin", "kna": "kena", "kne": "terlibat", "knn": "kanan", "knp": "kenapa", "knpa": "kenapa", "koar2": "berkata-kata", "kog": "kok", "kolot": "tua", "komen2nya": "komen-komennya", "komunitas2": "komunitas-komunitas", "konpers": "konferensi pers", "konseling2": "konseling-konseling", "kopdar": "kopi darat", "koplak": "kocak", "koplak2": "lucu-lucu", "koq": "kok", "korban2": "korban-korban", "kotak2": "kotak-kotak", "kpd": "kepada", "kpn": "kapan", "kpu": "komisi pemilihan umum", "kreta": "kereta", "krn": "karena", "krna": "karena", "krng": "kurang", "krupsi": "korupsi", "ksadaran": "kesadaran", "ksal": "kesal", "ksehtan": "kesehatan", "ksh": "kasih", "kt": "kita", "ktdkseimbangn": "ketidak seimbangan", "ktimpangan": "ketimpangan", "ktk": "ketika", "kuhp": "kitab undang-undang hukum pidana", "kwn": "kawan", "ky": "kaya", "kyai2": "kyai-kyai", "kyk": "kayak", "kyknya": "kayaknya", "kzl": "kesal", "laa": "lah", "lagi2": "lagi-lagi", "lagu2": "lagu-lagu", "lain2": "lain-lain", "laki2": "laki-laki", "laku2": "laku-laku", "lambai2": "lambai-lambai", "lang": "adil", "langi": "langit", "larang2": "larang-larang", "laskar2": "laskar-laskar", "lawyer2": "lawyer lawyer", "lbh": "lebih", "lebih2": "lebih-lebih", "lembaga2": "lembaga-lembaga", "lembu2": "lembu-lembu", "lepastu": "setelah itu", "lesbia": "lesbian", "let": "membiarkan", "lg": "lagi", "lgi": "lagi", "lh": "boleh", "lha": "lah", "lht": "lihat", "liat": "lihat", "liberalis2": "liberalis-liberalis", "like": "suka", "lirik2": "lirik-lirik", "live": "langsung", "liwat": "lewat", "lknt": "laknat", "lngsung": "langsung", "lo": "kamu", "loe": "lo", "lol": "lucu", "lom": "belum", "lope": "cinta", "lope2": "cinta-cinta", "love": "cinta", "lsm": "lembaga swadaya masyarakat", "lu": "kamu", "lwt": "lewat", "lyf": "live", "ma": "sama", "mabuhay": "hidup", "macam2": "macam-macam", "macem": "macam", "macem2": "macam-macam", "macet2an": "macet-macetan", "maen": "main", "majority": "mayoritas", "mak": "ibu", "mana2": "mana-mana", "mangkok2": "mangkok-mangkok", "manteman": "teman-teman", "marah2": "marah-marah", "maren2": "kemarin-kemarin", "mari": "ayo", "masama": "sama - sama", "masing2": "masih", "masy": "masyarakat", "masyrkt": "masyarakat", "mati2": "mati-mati", "mati2an": "mati-matian", "maybe": "mungkin", "mba": "mbak", "mbahas": "membahas", "mbak": "kaka", "mbela2": "membela-bela", "mbelain": "belain", "mcm": "macam", "mdh": "mudah", "mdh2an": "mudah-mudahan", "mean": "berarti", "medsos": "media sosial", "meet": "bertemu", "melayakni": "melayani", "meluk2": "peluk-peluk", "membuate": "membuat", "men": "laki-laki", "menag": "menteri agama", "mencak": "marah-marah", "mencak2": "marah-marah", "mencarut2": "mencarut", "mendpt": "mendapat", "menjdi": "menjadi", "menjelek2an": "menjelek-jelekan", "menklarifikasi": "mengklarifikasi", "menpora": "menteri pemuda dan olahraga", "menristek": "mentri riset, teknologi dan pendidikan", "mensos": "menteri sosial", "mentah2": "mentah-mentah", "mentang2": "mentang-mentang", "mentri": "menteri", "mereka2": "mereka-mereka", "mesjid": "masjid", "met": "selamat", "metro": "metropolitan", "mga": "moga", "mgkn": "mungkin", "mhn": "mohon", "milih2": "milih-milih", "miras": "minuman keras", "miumi": "majelis intelektual dan ulama muda indonesia", "mjadi": "menjadi", "mjadikan": "menjadikan", "mjd": "menjadi", "mkn": "makin", "mknn": "makanan", "mksd": "maksud", "mlalui": "melalui", "mlm": "malam", "mmang": "memang", "mmbawa": "membawa", "mmblhkan": "membolehkan", "mmbnrkan": "membenarkan", "mmbuat": "membuat", "mmg": "memang", "mmilih": "memilih", "mmng": "memang", "mna": "mana", "mndukung": "mendukung", "mngajarkan": "mengajarkan", "mninggal": "meninggal", "mnjd": "menjadi", "mnjdi": "menjadi", "mnjijikkan": "menjijikan", "mnolak": "menolak", "mnrtmu": "menurutmu", "mns": "manusia", "mnsia": "manusia", "mnt": "minta", "mnta": "minta", "mnyimpang": "menyimpang", "mnyumbang": "menyumbang", "mo": "mau", "moga": "semoga", "monday": "senin", "month": "bulan", "more": "lebih", "mosque": "masjid", "mostly": "kebanyakan", "move": "pindah", "mpr": "majelis permusyawaratan rakyat", "mprlihatkn": "memperlihatkan", "mreka": "mereka", "mrk": "mereka", "mrka": "mereka", "mrka2": "mereka-mereka", "msh": "masih", "msk": "masuk", "mslh": "masalah", "mslh2": "masalah-masalah", "msuh": "musuh", "msyarakat": "masyarakat", "mual": "mual mual", "mudah2an": "mudah-mudahan", "music": "musik", "muslims": "muslim-muslim", "muter2": "muter-muter", "mystery": "misteri", "n": "dan", "na": "nya", "nabi2": "nabi-nabi", "nak": "hendak", "name": "nama", "NaN": "dan", "nangis2": "nagis-nangis", "napsu": "nafsu", "napza": "narkoba psikotropika dan zat adiktif", "nasihat": "nasehat", "ndak": "indak", "negara2": "negara-negara", "negeri2": "negeri-negeri", "negri": "negeri", "neh": "nih", "neko": "macam", "new": "baru", "news": "berita", "nga": "enggak", "ngajak": "mengajak", "ngajak2": "ajak-ajak", "ngak": "enggak", "ngaku": "mengaku", "ngaku2": "akui-akui", "ngamuk2": "ngamuk-ngamuk", "ngedukung": "mendukung", "ngeliat": "melihat", "ngeri": "takut", "ngerti": "mengerti", "nggak": "enggak", "ngguyu": "ketawa", "ngguyu2": "ketawa-ketawa", "ngibul": "bohong", "ngn": "dengan", "ngo": "non governmental organization", "ngrasa": "merasa", "ngusir": "usir", "ngusir2": "usir-usir", "ngutip2": "ngutip-ngutip", "nh": "nih", "ni": "nih", "nice": "bagus", "nie": "nih", "night": "malam", "nilai2": "nilai-nilai", "nitizen": "penonton", "nk": "ingin", "nma": "nama", "nmn": "namun", "nmpk": "nampak", "nnt": "nanti", "nnti": "nanti", "no": "tidak", "norma2": "norma-norma", "note": "catat", "now": "sekarang", "ntap": "mantap", "ntar": "entar", "ntt": "nusa tenggara timur", "nu": "nahdatul ulama", "nunggu": "menunggu", "nuntut2": "nuntut-nuntut", "nyampyr": "nyampur", "nyata2": "nyata-nyata", "nyatakn": "nyatakan", "nyela2": "nyela-nyela", "nympr": "nyampur", "nyuruh": "suruh", "nyuruh2": "suruh-suruh", "o": "oleh", "obok2": "aduk-aduk", "ocehan2nya": "omongan-omongannya", "odmk": "orang dengan masalah kejiwaan", "oia": "oh iya", "oknum2": "oknum oknum", "ol": "oleh", "olh": "oleh", "olok2": "olok-olok", "omg": "oh my god", "omong2": "ngomong-ngomong", "opini2": "opini-opini", "opinion": "opini", "or": "orang", "orang": "manusia", "orang2": "orang-orang", "orang2nya": "orang-orangnya", "org": "orang", "org2": "orang-orang", "org2nya": "orang-orangnya", "orglaen": "orang lain", "orgnya": "orangnya", "orgullo": "kebangaan", "ormas2": "ormas-ormas", "orng": "orang", "orng2": "orang-orang", "ortu": "orang tua", "otak2nya": "otak-otaknya", "other": "lainnya", "our": "kami", "p": "per", "p.s": "pesan", "p0rn": "porno", "pabji": "pubg", "pake": "pakai", "panjang2": "panjang-panjang", "partai2": "partai-partai", "partner": "pasangan", "pas": "ketika", "pasal2": "pasal-pasal", "paslon": "pasangan calon", "pbnu": "pengurus besar nahdatul ulama", "pd": "percaya diri", "pdhal": "padahal", "pdhl": "padahal", "pdip": "partai demokrasi Indonesia perjuangan", "pdskji": "perhimpunan dokter spesialis kedokteran jiwa indonesia", "pdt": "padat", "pean": "anda", "pecahan2": "pecahan-pecahan", "pedofili": "pedofil", "pejabat2": "pejabat-pejabat", "pelaku2": "pelaku-pelaku", "pelan2": "pelan-pelan", "pelanggarab": "pelanggaran", "pelcehn": "pelecehan", "peliculas": "film", "pelik2": "pelik=pelik", "pelintir2": "pelintir-pelintir", "pembela2": "pembela-pembela", "pemilu": "pemilihan umum", "pemko": "pemerintah kota", "pemprov": "pemerintah provinsi", "pendahulu2nya": "pendahulu-pendahulunya", "pendukung2": "pendukung-pendukung", "pengaruh2": "pengaruh-pengaruh", "pengen": "pengin", "pengidap2": "pengidap-pengidap", "penyerangan2": "penyerangan-penyerangan", "penyimpangan2": "penyimpangan-penyimpangan", "people": "orang", "peremp": "perempuan", "perilaku2": "perilaku-perilaku", "perkahwinan": "perkawinan", "person": "orang", "perusahaan2": "perusahaan-perusahaan", "pgi": "pagi", "photo": "foto", "pihak2": "pihak pihak", "pilih2": "pilih-pilih", "pink": "merah muda", "pinter2": "pintar-pintar", "piye": "bagaimana", "pk": "pakai", "pkb": "partai kebangkitan bangsa", "pke": "pakai", "pket": "paket", "pki": "partai komunis indonesia", "pkl": "pukul", "pkr": "pakar", "pksejahtera": "pks", "please": "tolong", "plis": "please", "pls": "tolong", "plsrt": "please retweet", "pmhaman": "pemahaman", "pmimpin": "pemimpin", "pmmpin": "pemimpin", "pndkung": "pendukung", "pndokong": "pendukung", "pngikut2mu": "pengikut-pengikutmu", "pnjilat": "penjilat", "pns": "pegawai negeri sipil", "pny": "punya", "pnyokong": "penyokong", "poin2": "poin-poin", "pon": "juga", "poster2": "poster-poster", "pp": "pimpinan pusat", "ppdgj": "pedoman penggolongan diagnosis gangguan jiwa", "presenter": "pembawa acara", "presenter2": "pembawa acara - pembawa acara", "pride": "kebanggaan", "prilaku": "perilaku", "prnh": "pernah", "produk2": "produk-produk", "program2": "program-program", "proud": "bangga", "prov": "provinsi", "provokasi2": "provokasi-provokasi", "psl": "pasal", "psr": "pasar", "ptemuan": "pertemuan", "puak2": "suku-suku", "public": "publik", "pubg": "player unknown's battlegrounds", "puji2": "puji-puji", "pulak": "pula", "pura2": "pura-pura", "pusat2": "pusat-pusat", "puskesmas2": "puskesmas-puskesmas", "q": "aku", "qorban": "qurban", "qt": "kita", "queen": "ratu", "rabb": "Allah", "rainbow": "pelangi", "rakan": "teman", "ramai2": "ramai-ramai", "rang": "orang", "ranjau2": "ranjau-ranjau", "rasuah": "korupsi", "rata2": "rata-rata", "read": "baca", "red": "merah", "rekan2": "rekan-rekan", "rempah2": "rempah-rempah", "report": "laporan", "research": "penelitian", "respect": "menghormati", "reunion": "reuni", "review": "ulasan", "ri": "republik Indonesia", "ribut2": "ribut-ribut", "rights": "benar", "rumah2": "rumah-rumah", "ruu": "rancangan undang-undang", "rwt": "rawat", "sah2": "sah-sah", "sahabat2": "sahabat sahabat", "salam2": "salam-salam", "salamat": "selamat", "sama2": "sama-sama", "sampe": "sampai", "sarap": "gila", "satpam": "satuan pengamanan", "satu2": "satu-satu", "saturday": "sabtu", "saudara2": "saudara-saudara", "save": "menyimpan", "sbaliknya": "sebaliknya", "sbb": "sebab", "sbg": "sebagai", "sbgai": "sebagai", "sbgian": "sebagian", "sbgn": "sebagian", "sblm": "sebelum", "sblmnya": "sebelumnya", "sbnr": "sebenar", "sbntr": "sebentar", "scr": "secara", "scra": "secara", "sdg": "sedang", "sdh": "sudah", "sdkt": "sedikit", "sdri": "sendiri", "se": "surat edaran", "se7": "setuju", "seakan2": "seakan akan", "sebaik2": "sebaik-sebaik", "sebenernya": "sebenarnya", "sebgmn": "sebagaimana", "sebuk": "sibuk", "sedar": "sadar", "see": "sih", "sehari2": "sehari-hari", "selaen": "selain", "selamet": "selamat", "selmt": "selamat", "seluas2nya": "seluas-luasnya", "semata2": "semata-mata", "sendiri\u201d": "sendiri sendiri", "seneng": "senang", "seolah2": "seolah-olah", "seorg": "seorang", "separah2nya": "separah-parahnya", "serem": "seram", "sering2": "sering-sering", "serious": "serius", "seronok": "menyenangkan", "sesok": "besok", "set": "setengah", "setengah2": "setengah-setengah", "setting": "pengaturan", "sgl": "segala", "sgrc": "support group and resource center", "sgrlah": "segeralah", "sgt": "sangat", "she": "dia", "shg": "sehingga", "show": "menunjukkan", "shrsnya": "seharusnya", "siap2": "siap-siap", "siapa2": "siapa-siapa", "sigh": "sih", "sik": "sih", "simbol2": "simbol-simbol", "sis": "kakak", "sista": "kakak", "sister": "kakak", "sisters": "kakak-kakak", "situs2": "situs-situs", "sj": "saja", "sjk": "sejak", "skali": "sekali", "skarang": "sekarang", "skg": "sekarang", "skr": "sekarang", "skrg": "sekarang", "skrng": "sekarang", "slh": "salah", "sll": "selalu", "sllu": "selalu", "slmet": "selamat", "slmt": "selamat", "slow2": "santai-santai", "sm": "sama", "smakin": "semakin", "smart": "pintar", "smg": "semoga", "smkn": "semakin", "smentara": "sementara", "smoga": "semoga", "smpe": "sampai", "smtr": "semester", "smua": "semua", "smw": "semua", "sndiri": "sendiri", "sndri": "sendiri", "sngt": "sangat", "soal2": "soal-soal", "social": "sosial", "society": "masyarakat", "socmed": "sosmed", "sok": "berlaga", "sok2": "sok-sok", "sok2an": "sok-sokan", "sokong": "dukung", "sono": "sana", "sorry": "maaf", "space": "ruang", "special": "spesial", "sperti": "seperti", "sprt": "seperti", "sprti": "seperti", "spt": "seperti", "sptnya": "sepertinya", "spy": "supaya", "srt": "surat", "starbucks": "starbuck", "statement": "pernyataan", "stengah": "setengah", "sticker2": "stiker-stiker", "still": "tetap", "stlh": "setelah", "stop": "berhenti", "story": "cerita", "straight": "lurus", "student": "pelajang", "suap2an": "suap-suapan", "subsidi2": "subsidi-subsidi", "suka2": "suka-suka", "suku2": "suku-suku", "sunday": "minggu", "sungguh": "sungguh sungguh", "support": "mendukung", "surat2an": "surat-suratan", "surprise": "kejutan", "survey": "survei", "sweeping": "luas", "sy": "saya", "sya": "saya", "syariah2an": "syariah-syariahan", "syg": "sayang", "tahu2": "tau-tau", "tai2nya": "tai-tainya", "tajuk2": "tajuk-tajuk", "tak": "tidak", "takde": "tidak ada", "takut2": "takut-takut", "talking": "pembicaraan", "tampa": "tanpa", "tanda2": "tanda-tanda", "tanda2nya": "tanda-tandanya", "tanya2": "tanya-tanya", "tarap": "taraf", "tarik2": "tarik-tarik", "tau": "tahu", "tau2": "tau-tau", "taun": "tahun", "tayangan2": "tayangan-tayangan", "tbh": "tubuh", "td": "tadi", "tdk": "tidak", "team": "tim", "teken": "tekan", "teman2": "teman-teman", "temen": "teman", "temen2": "teman-teman", "temple": "candi", "tenang2": "tenang tenang", "tengah2": "tengah-tengah", "terang2": "terang-terang", "terang2an": "terang-terangan", "tereak2": "teriak-teriak", "tereiak2": "teriak-teriak", "terheran2": "terheran-heran", "teriak2": "teriak-teriak", "terong2an": "terong-terongan", "tetep": "tetap", "tetp": "tetap", "teve": "televisi", "tgan": "tangan", "tggl": "tanggal", "tgk": "tengok", "tgok": "tengok", "th": "tahu", "thanks": "terima kasih", "that": "itu", "thd": "terhadap", "thdp": "terhadap", "their": "mereka", "then": "kemudian", "they": "mereka", "think": "berfikir", "this": "ini", "thn": "tahun", "those": "itu", "thx": "thanks", "tiap2": "tiap-tiap", "tiba2": "tiba-tiba", "tide": "pasang", "tim2": "tim-tim", "time": "waktu", "tinggi2": "tinggi-tinggi", "tipi": "televisi", "tiru2": "tiru tiru", "tk": "tak", "tkde": "tidak ada", "tl": "timeline", "tlp": "telepon", "tmn": "teman", "tmn2": "teman-teman", "tmnnya": "temannya", "tmpt": "tempat", "tni": "tentara nasional indonesia", "tnp": "tanpa", "tntg": "tentang", "tnya": "tanya", "today": "hari ini", "tokoh2": "tokoh-tokoh", "tp": "tapi", "tpi": "tapi", "trfkir": "terfikir", "trh": "terhadap", "trhdp": "terhadap", "trjdi": "terjadi", "trkhr": "terakhir", "trllu": "terlalu", "trmsk": "termasuk", "trs": "terus", "true": "benar", "trus": "terus", "tryt": "ternyata", "tsb": "tersebut", "ttg": "tentang", "ttng": "tentang", "ttp": "tetap", "tu": "itu", "tuh": "itu", "tuk": "untuk", "tulisan2mu": "tulisan-tulisan mu", "turun2": "turun-turun", "tv": "televisi", "twit": "tweet", "twit2": "tweet-tweet", "twitwat": "twitter", "twt": "tweet", "type": "tipw", "type2": "tipe-tipe", "ubun2": "ubun-ubun", "ud": "sudah", "udah": "sudah", "udh": "sudah", "ugm": "universitas gajah mada", "ui": "universitas indonesia", "ujung2": "ujung-ujung", "ujung2nya": "ujung-ujungnya", "unair": "universitas airlangga", "unand": "universitas andalas", "undang2": "undang undang", "univ": "universites", "unsur2": "unsur-unsur", "unt": "untuk", "untk": "untuk", "usir2": "usir-usir", "ust": "ustadz", "ut": "untuk", "utk": "untuk", "uu": "undang-undang", "uud": "undang-undang dasar", "very": "sangat", "via": "melalui", "view": "melihat", "virus2": "virus-virus", "voice": "suara", "w": "gue", "wapres": "wakil presiden", "warbiyasak": "luar biasa", "waria": "wanita pria", "watch": "menonton", "weekend": "hari libur", "what": "apa", "who": "world health organization", "why": "kenapa", "wise": "bijaksana", "wish": "harapan", "with": "dengan", "wkt": "waktu", "world": "dunia", "wota2": "wota-wota", "wrong": "salah", "x": "kali", "xan": "tak akan", "xew": "apa", "yak": "ya", "ybs": "yang bersangkutan", "yesterday": "kemarin", "yg": "yang", "you": "anda", "yowes": "yasudah", "yowis": "yasudah", "yrs": "iya", "yuk": "ayo", "yukk": "yuk", "yuu": "yuk", "yuuk": "yuk", "zero": "nol", "zholim": "dzalim", "zinah": "zina", "zine": "zina", "zionist": "zionis", "woww": "wow", "aminn": "amin", "netaas": "menetas", "keberpa": "keberapa", "eeeehhhh": "eh", "kata2nyaaa": "kata-katanya", "hallo": "halo", "kaka": "kakak", "ka": "kak", "daah": "deh", "aaaaahhhh": "ah", "yaa": "ya", "smga": "semoga", "slalu": "selalu", "amiin": "amin", "kk": "kakak", "sii": "sih", "nyenengin": "menyenangkan", "gemess": "gemas", "akuuu": "aku", "gemesiin": "menggemaskan", "menyenangkn": "menyenangkan", "rb": "ribu", "akau": "aku", "saranin": "sarankan", "nemuin": "menemukan", "yah": "ya", "mnis2nya": "manis-manisnya", "gtu": "begitu", "idolaa": "idola", "bangett": "banget", "kaakk": "kak", "baaiikkk": "baik", "telaatt": "telat", "kaak": "kak", "sempet": "sempat", "karna": "karena", "ngga": "enggak", "mantep": "mantap", "loohhh": "loh", "yeee": "ya", "baguuuuus": "bagus", "ngikutin": "mengikuti", "gayany": "gayanya", "bnk": "bang", "awaas": "awas", "siip": "sip", "abizzzz": "habis", "kerrrrennnn": "keren", "sayaa": "saya", "kpan": "kapan", "sma": "sama", "sdkit": "sedikit", "bngt": "banget", "lgu": "lagu", "kharismanya": "karismanya", "lgunya": "lagunya", "kereeeeennnn": "keren", "nyamain": "menyamakan", "lakik": "laki", "siippp": "sip", "nyangka": "menyangka", "bangettt": "banget", "kereeen": "keren", "wihhhhhh": "wih", "inget": "ingat", "bingitsss": "banget", "dmn": "dimana", "bird": "biru", "kaloo": "kalo", "gimna": "bagaimana", "tuuh": "tuh", "nyebur": "mencebur", "yng": "yang", "dbuat": "dibuat", "kn": "kan", "pingin": "pengin", "bangcat": "bangsat", "yha": "ya", "waoohhh": "wah", "pngn": "pengin", "dhe": "deh", "indahhh": "indah", "bgttt": "banget", "tuhh": "tuh", "berarto": "berarti", "aamiin": "amin", "amiiinn": "amin", "aminnnn2": "amin-amin", "amiiiin": "amin", "aminnn": "amin", "amiend": "amien", "yaaa": "ya", "ameen": "amin", "aminnnn": "amin", "amiiinnn": "amin", "ngomong": "mengomong", "mamer": "memamerkan", "keles": "kali", "jelase": "jelas", "uwakk": "uwak", "gini": "begini", "salfok": "salah fokus", "njir": "anjir", "piiink": "pink", "astgfrullah": "astagfirullah", "ko": "kok", "cd": "celana dalam", "gbs": "enggak bisa", "bhs": "bahasa", "yaaaaaa": "ya", "bangshad": "bangsat", "stagahhhh": "astaga", "iooo": "lo", "plampiasannn": "pelampiasan", "masalahh": "masalah", "tohh": "toh", "inghinn": "ingin", "dhi": "di", "mengherthii": "mengerti", "yanckh": "yang", "chelalyyu": "selalu", "therchakitty": "tersakiti", "mauuuuu": "mau", "kaliii": "kali", "bangat": "banget", "bkin": "bikin", "dongg": "dong", "yokkk": "yuk", "yu": "yuk", "sweger": "segar", "gantengg": "ganteng", "bingiit": "banget", "kebukaaa": "terbuka", "keliatan": "kelihatan", "yakk": "ya", "kayakk": "kayak", "asliii": "asli", "cntiknya": "cantiknya", "bat": "banget", "ngeliatnya": "melihatnya", "njerr": "anjir", "bingiiittttzzzz": "banget", "dooong": "dong", "masi": "masih", "gni": "begini", "hdk": "hendak", "jugaa": "juga", "mksud": "maksud", "mbakx": "mbaknya", "syantik": "cantik", "kite": "kita", "pa": "apa", "bejilbbnya": "berjilbabnya", "masio": "masih", "yahhh": "ya", "makasih": "terima kasih", "leh": "boleh", "ugaaa": "juga", "jerewi": "jerawat", "ilang": "hilang", "jilbabnyaah": "jilbabnya", "yuuuk": "yuk", "warnaaa": "warna", "punyaaa": "punya", "cantiik": "cantik", "sukaaak": "suka", "kebayanyaa": "kebayanya", "mantes": "memantas", "merekaaaa": "mereka", "jugaaa": "juga", "gilaaaa": "gila", "tiga2nya": "tiga-tiganya", "lucuu": "lucu", "emessss": "gemas", "ucullllll": "lucu", "males": "malas", "bangeettt": "banget", "gemesshh": "gemas", "apaaa": "apa", "ampuunn": "ampun", "ciuuuummmm": "cium", "mbil": "sambil", "ngmng": "mengomong", "ngemeng": "mengomong", "epe": "apa", "seehh": "sih", "yank": "sayang", "ucul": "lucu", "yahh": "ya", "ngakak": "mengakak", "astagfirrullah": "astagfirullah", "suerrr": "suer", "ntik": "cantik", "dapet": "dapat", "suru": "suruh", "lucuuuu": "lucu", "ketawakkkk": "ketawa", "kyanya": "kayanya", "bnget": "banget", "nakutin": "menakutkan", "malingg": "maling", "cocokk": "cocok", "niihh": "nih", "imutttttt": "imut", "satuuuuu": "satu", "sesekk": "sesak", "nafass": "nafas", "bgtt": "banget", "cintaaaaaaa": "cinta", "lucuk": "lucu", "duluu": "dulu", "megang": "memegang", "kawinin": "mengawini", "liatin": "perlihatkan", "ny": "nya", "bdan": "badan", "nyaaa": "nya", "gemesin": "menggemaskan", "fotoin": "memfoto", "kaaa": "kak", "pcarny": "pacarnya", "kri": "kiri", "pda": "pada", "cmn": "cuman", "garuk2": "garuk-garuk", "tembokk": "tembok", "namax": "namanya", "kiriii": "kiri", "bungkusin": "bungkuskan", "atu": "satu", "dehhhh": "deh", "kakk": "kakak", "ngegantiin": "menggantikan", "alloh": "allah", "trsa": "terasa", "khadirannya": "kehadirannya", "fikiran": "pikiran", "hayalanku": "khayalanku", "bru": "baru", "nton": "menonton", "ngapain": "mengapa", "surgaa": "surga", "kangenn": "kangen", "kgn": "kangen", "kakkkkk": "kak", "rinduuuuuuuuu": "rindu", "ninggalin": "meninggalkan", "terangkhn": "terangkan", "bgtttt": "banget", "slama": "selama", "bngunlah": "bangunlah", "bngun": "bangun", "ea": "ya", "qu": "ku", "hiri": "hari", "teman\"ny": "teman-temannya", "aza": "saja", "alm": "almarhum", "sie": "sih", "benet": "benar", "bangetttt": "banget", "tuu": "tuh", "skrang": "sekarang", "gda": "enggak ada", "gantiin": "menggantikan", "kta": "kita", "kklau": "kalau", "acra": "acara", "kasian": "kasihan", "yaaa ": "ya", "bnerr": "benar", "emk": "emak", "temenn": "teman", "pngenn": "pengin", "terkenall": "terkenal", "ajhh": "saja", "sungguh2": "sungguh-sungguh", "mnjadi": "menjadi", "shbt": "sahabat", "bner": "benar", "ktny": "katanya", "dimusiumin": "dimuseumkan", "bntu": "bantu", "kangem": "kangen", "amiiin": "amin", "mudh\"an": "mudah-mudahan", "tenny": "tenang", "lahhh": "lah", "maap": "maaf", "mnrtkuu": "menurutku", "ikutin": "ikuti", "jdwl": "jadwal", "wajiiib": "wajib", "aaaammmiiiinnnn": "amin", "dtnggu": "ditunggu", "kunjungin": "kunjungi", "lagiiii": "lagi", "waktunyaaa": "waktunya", "diluangin": "diluangkan", "jga": "juga", "mimin": "admin", "istimewaaa": "istimewa", "blom": "belum", "nangis": "menangis", "ngaku\"": "mengaku-ngaku", "diaa": "dia", "donkk": "dong", "kejawab": "terjawab", "yaaaa": "ya", "seru2an": "seru-seruan", "mohom": "mohon", "kebalas": "terbalas", "nyempetin": "menyempatkan", "wktunya": "waktunya", "kaaaa": "kak", "ngobrol": "mengobrol", "namain": "namai", "ijap": "ijab", "kobul": "kabul", "co": "cowok", "pst": "pasti", "inggat": "ingat", "brdua": "berdua", "sanggat": "sangat", "siipp": "sip", "hdup": "hidup", "kedpan": "kedepan", "mndaptkan": "mendapatkan", "nutup\"in": "menutup-nutupi", "iyah": "iya", "sja": "saja", "kakkk": "kak", "za": "saja", "mulut2": "mulut-mulut", "peengn": "pengin", "tuch": "tuh", "muantteepp": "mantap", "cpt": "cepat", "mrit": "merit", "y": "nya", "ngapasih": "mengapa sih", "segitunya": "sebegitunya", "fto": "foto", "sisain": "menyisakan", "slamt": "selamat", "nantiii": "nanti", "edaaan": "edan", "tunggu2": "tunggu-tunggu", "sehat2": "sehat-sehat", "ciyeeeeeee": "cie", "pantes": "pantas", "lairannya": "kelahirannya", "jmnya": "jamnya", "ajha": "saja", "nyaaaa": "nya", "nyari": "mencari", "amiien": "amien", "terbaikkk": "terbaik", "ayahh": "ayah", "ae": "saja", "mrekaaaa": "mereka", "yaampuuuunnnn": "ya ampun", "nyanyiin": "menyanyi", "ahhh": "ah", "pgen": "pengin", "tempt": "tempat", "kh": "kah", "bgini": "begini", "brani": "berani", "asyikkkkk": "asyik", "iriiiii": "iri", "naaaaahh": "nah", "kka": "kakak", "ttep": "tetap", "jrang": "jarang", "msuk": "masuk", "gossip2.an": "gosip-gosipan", "makkk": "mak", "guee": "gue", "pengennn": "pengin", "gilzz": "gila", "nyaru": "menyaru", "gilaaa": "gila", "kebarat2an": "kebarat-baratan", "ohh": "oh", "mantabs": "mantab", "jgan": "jangan", "mlu": "melulu", "payahh": "payah", "ntaf": "mantap", "beneran": "benaran", "mudah\"n": "mudah-mudahan", "milikin": "memiliki", "kamo": "kamu", "inih": "ini", "sukaaaa": "suka", "sukaaaaaa": "suka", "kirain": "mengira", "mehoonggggg": "mahal", "apikkkk": "apik", "yach": "ya", "kereenn": "keren", "ginii": "begini", "ayoook": "ayo", "yoiii": "yoi", "gituuu": "begitu", "iyaaa": "iya", "pgn": "pengin", "liptsk": "lipstik", "nyeplak": "menjiplak", "cw": "cewek", "kmarin": "kemarin", "mengncang": "mengencang", "tauu": "tau", "mksh": "terima kasih", "skitar": "sekitar", "ktmu": "ketemu", "tmen": "teman", "heii": "hai", "diputusin": "diputuskan", "pcrnya": "pacarnya", "mnjual": "menjual", "ngadain": "mengadakan", "kkak": "kakak", "rekomendasiin": "rekomendasikan", "sihhh": "sih", "cooocok": "cocok", "mrip": "mirip", "mama.nyy": "mamanya", "bosenin": "membosankan", "kmu": "kamu", "sey": "sih", "cantikkk": "cantik", "banger": "banget", "kaa": "kak", "eakyu": "waktu", "cntik": "cantik", "kakanya": "kakaknya", "doeloe": "dulu", "cantikkkk": "cantik", "bngtt": "banget", "iyak": "iya", "malh": "malah", "mlah": "malah", "gag": "enggak", "fhotonya": "fotonya", "bleh": "boleh", "tnggl": "tanggal", "tanteny": "tantenya", "sklinyaa": "sekalinya", "ayoo": "ayo", "dramaa": "drama", "niiih": "nih", "bisaa": "bisa", "dunk": "dong", "lihaat": "lihat", "yuwkkk": "yuk", "editannn": "editan", "mw": "mau", "thu": "itu", "mai": "mau", "kamuu": "kamu", "pastii": "pasti", "bisaaa": "bisa", "kejuu": "keju", "kereen": "keren", "nama.a": "namanya", "kmauan": "kemauan", "ahhhh": "ah", "yuuuuk": "yuk", "dmana": "dimana", "disiniii": "disini", "cobaa": "coba", "bayangin": "bayangkan", "yaudh": "ya sudah", "dl": "dulu", "ajaaa": "saja", "tuhhh": "tuh", "amploppp": "amplop", "sayangg''": "sayang-sayang", "nantii": "nanti", "aminnnnn": "amin", "gakuat": "enggak kuat", "yukkkk": "yuk", "nged": "banget", "nantik": "nanti", "ksitu": "kesitu", "bangeett": "banget", "liad": "lihat", "biasahhh": "biasa", "tutorialx": "tutorialnya", "dpke": "dipakai", "fhoto": "foto", "kerenn": "keren", "binggo": "banget", "rekomndd": "rekomendasi", "bli2n": "beli-belikan", "wado": "waduh", "guaa": "gua", "yaakkk": "ya", "bangetttttyt": "banget", "bangettttt": "banget", "bangg": "bang", "kangeeeeeeeeennnnnnnn": "kangen", "kaliannnnnn": "kalian", "rinduuuuuuuuuu": "rindu", "kangennnnnmnnnnnnnnnnnnnnnnn": "kangen", "kaliannnnnm": "kalian", "kangeeeeeeeeeeennnnn": "kangen", "kangennnnnnnnnnnnnnnn": "kangen", "kangennn": "kangen", "kaliannnnnnn": "kalian", "th2": "tahun-tahun", "smpt": "sempat", "trserah": "terserah", "prnah": "pernah", "wlwpun": "walaupun", "trpsah": "terpisah", "prsatukn": "persatukan", "ngomng": "mengomong", "skedar": "sekadar", "kangeeerrn": "kangen", "terrrrbaik": "terbaik", "jorookknya": "joroknya", "ihhhhh": "ih", "padahall": "padahal", "jorokkkk": "jorok", "mualll": "mual", "klow": "kalau", "muke": "muka", "ajh": "saja", "taiiii": "tahi", "seneeeeeng": "senang", "nyesel": "menyesal", "besarin": "membesarkan", "tanyang": "tayang", "ber2": "berdua", "jngan": "jangan", "ngelarang": "melarang", "kerreeen": "keren", "pantess": "pantas", "bngettt": "banget", "baperr": "bawa perasaan", "sudh": "sudah", "sbr": "sabar", "pngen": "pengin", "nntn": "menonton", "ht": "hati", "pemainy": "pemainnya", "aqu": "aku", "nyampe": "sampai", "kbawa": "terbawa", "bradegan": "beradegan", "amiiinnnn": "amin", "nikahin": "menikahi", "hadeuh": "aduh", "nyambung": "menyambung", "liyat": "lihat", "sk": "suka", "adengan": "adegan", "bingit": "banget", "dapetin": "mendapatkan", "model2": "model-model", "sitayngin": "ditayangkan", "cocokkk": "cocok", "mahhhh": "mah", "cocokny": "cocoknya", "iniw": "ini", "setju": "setuju", "bnern": "benaran", "pcrn": "pacaran", "klian": "kalian", "cocokkkk": "cocok", "bngett": "banget", "abes": "habis", "br": "baru", "mksdny": "maksudnya", "brpa": "berapa", "ihh": "ih", "kesampean": "kesampaian", "beginiiii": "begini", "ntr": "entar", "beliin": "belikan", "kya": "kaya", "keq": "kayak", "ngerasa": "merasa", "sygkuu": "sayangku", "ngeselin": "mengesalkan", "baju'a": "bajunya", "gara''": "gara-gara", "teruss": "terus", "pokok'a": "pokoknya", "ubur2": "ubur-ubur", "kuyy": "yuk", "bentarr": "bentar", "dureen": "duren", "ajibb": "ajib", "mahall": "mahal", "niiihh": "nih", "imuuttt": "imut", "imuttzz": "imut", "nha": "nya", "konten2": "konten-konten", "rasain": "merasai", "daaaah": "deh", "asliiii": "asli", "cantiknnnnyaaaaaaa": "cantiknya", "ngelihatnya": "melihatnya", "bdn": "badan", "lau": "kau", "cantikk": "cantik", "sexxi": "seksi", "bulet": "bulat", "ituu": "itu", "ngocok": "mengocok", "nyoba": "mencoba", "gn": "begini", "yaahhh": "ya", "kencqng": "kencang", "kra": "kira", "diginikan": "dibeginikan", "coret\"": "coret-coret", "lahh": "lah", "gambarin": "gambarkan", "ngerii": "ngeri", "digambarin": "digambarkan", "klen": "kalian", "bangke": "bangkai", "liatnya": "lihatnya", "bedanyaaa": "bedanya", "mbk": "mbak", "pgin": "pengin", "krudungan": "kerudungan", "ajaa": "saja", "maksutku": "maksudku", "kerennnnn": "keren", "njirr": "anjir", "jd'y": "jadinya", "manisnyaaaa": "manisnya", "thun": "tahun", "mbg": "mbak", "bangeet": "banget", "ngaco": "mengaco", "ngomen": "mengomentari", "kaaakk": "kak", "gituh": "begitu", "tenggah": "tengah", "uuuuuaaaaaaaaaappppppppppiiiiiiiiiiiiiiiiiikkkkkkkkkkkkkkkk": "apik", "kocakkkkkkkkkkkkkk": "kocak", "ksukaanmuuu": "kesukaanmu", "bagiii": "bagi", "ayooo": "ayo", "nontonx": "menontonnya", "pi": "tapi", "penasarannnn": "penasaran", "gilakkkkkk": "gila", "mauuu": "mau", "nontoonnn": "menonton", "cepet2": "cepat-cepat", "kepingin": "pengin", "nontonnnn": "menonton", "deeek": "dek", "ntappp": "mantap", "deg2an": "deg-degan", "cyiiin": "cinta", "iyha": "iya", "kureeeen": "keren", "muantaap": "mantap", "ngantri": "mengantri", "bli": "beli", "nraktir": "mentraktir", "nonton": "menonton", "sbnyak": "sebanyak", "penasaraaann": "penasaran", "ayoooo": "ayo", "nii": "ini", "nontonya": "menontonnya", "qta": "kita", "peen": "pengin", "lw": "kalau", "gga": "enggak", "yaah": "ya", "asikkk": "asik", "gilak": "gila", "ceritax": "ceritanya", "yoooiii": "yoi", "kuy": "yuk", "udahh": "sudah", "tauuuuu": "tau", "sabarrrrr": "sabar", "kqmu": "kamu", "banggain": "banggakan", "pengan": "pengin", "sendeng": "senang", "simpen": "simpan", "gakk": "enggak", "bagiin": "membagikan", "nanggepin": "menanggapi", "kapn": "kapan", "barang2": "barang-barang", "kirimin": "mengirimi", "senengin": "senangi", "ngasih": "mengasih", "ampunnnn": "ampun", "senengg": "senang", "teruzzz": "terus", "mantabb": "mantab", "sumpahh": "sumpah", "diluarann": "diluaran", "trbaik": "terbaik", "dlam": "dalam", "anjirrrr": "anjir", "jendess": "janda", "sichh": "sih", "namnya": "namanya", "geh": "gih", "22nya": "dua-duanya", "aduuhhh": "aduh", "miripp": "mirip", "silauuu": "silau", "waww": "wow", "uga": "juga", "njiirr": "anjir", "kesinii": "kesini", "gemay": "gemas", "dstu": "disitu", "wiiiihhhh": "wih", "samoga": "semoga", "kereennn": "keren", "wahh": "wah", "nyangkaa": "menyangka", "hasilnyaaa": "hasilnya", "dahhhh": "dah", "mudah2": "mudah-mudah", "jugaakk": "juga", "diapa2in": "diapa-apai", "yaak": "ya", "lhoo": "loh", "donx": "dong", "gthu": "begitu", "nda": "indak", "maauuu": "mau", "pacarrr": "pacar", "idamann": "idaman", "naturalllll": "natural", "bnyak": "banyak", "berntung": "beruntung", "merka": "mereka", "jln2": "jalan-jalan", "romntis": "romantis", "slamat": "selamat", "ulan": "ulang", "mngkin": "mungkin", "diksi": "dikasih", "sipp": "sip", "nungguin": "menunggu", "ngeluarin": "mengeluarkan", "keciill": "kecil", "ughaa": "juga", "tante\"": "tante-tante", "banged": "banget", "abeees": "habis", "kapan2": "kapan-kapan", "adain": "mengadakan", "bosen2": "bosan-bosan", "sempurnaaaaa": "sempurna", "kaan": "kan", "kece2": "kece-kece", "badaaiii": "badai", "beneraan": "benaran", "cantiiik": "cantik", "kuatttt": "kuat", "cantiiiiiiik": "cantik", "cantiknyaaa": "cantiknya", "bae2": "baik-baik", "bgd": "banget", "mb": "mbak", "catiiiiik": "cantik", "subahanallah": "subhanallah", "jaman2nya": "jaman-jamannya", "padet": "padat", "buktiin": "membuktikan", "cantiq": "cantik", "pasien\"nya": "pasien-pasiennya", "lohh": "lo", "keeeerrrreeeennnnnnnnn": "keren", "woow": "wow", "choker2": "choker-choker", "hadirrr": "hadir", "hadirr": "hadir", "jelashh": "jelas", "sekali2": "sekali-kali", "mbaaa": "mbak", "hidupppp": "hidup", "nyebelin": "menyebalkan", "diliatnya": "dilihatnya", "abis2": "habis-habis", "para2": "parah-parah", "bangeeet": "banget", "allahhhhhhh": "allah", "gemesnya": "gemasnya", "ilang2": "hilang-hilang", "kmi": "kami", "ngilangin": "menghilangkan", "tbuh": "tubuh", "itungan": "hitungan", "ledekin": "ledeki", "repot2": "repot-repot", "oles2": "oles-oles", "ckup": "cukup", "dptkan": "dapatkan", "kalii": "kali", "ninggiin": "meninggikan", "baget": "banget", "lgsg": "langsung", "ngk": "enggak", "ahh": "ah", "pas2an": "pas-pasan", "balesin": "membalas", "gausah": "enggak usah", "sukaaa": "suka", "lutuna": "lucunya", "ciapa": "siapa", "gemesss": "gemas", "duhh": "duh", "terharuuu": "terharu", "alamii": "alami", "sampek": "sampai", "ckrng": "sekarang", "ngegemesin": "menggemaskan", "imoet": "imut", "imut2": "imut-imut", "jagain": "menjaga", "yahbh": "ya", "msih": "masih", "neko2": "neko-neko", "sktika": "seketika", "lutu": "lucu", "wowwww": "wow", "massss": "mas", "kesiniii": "kesini", "keren\"": "keren-keren", "jalan\"nya": "jalan-jalannya", "masyaallahhh": "masyaallah", "kerennn": "keren", "ksni": "kesini", "begok": "bego", "bkan": "bukan", "bawak": "bawa", "it": "itu", "nti": "nanti", "keeeeeren": "keren", "baguusss": "bagus", "nahannnn": "tahan", "bagaatt": "banget", "gayannxa": "gayanya", "kereeeen": "keren", "uda": "sudah", "ngekeh": "kekeh", "xlian": "kalian", "b3": "bertiga", "ihhh": "ih", "lcu": "lucu", "pait": "pahit", "koplakkk": "koplak", "kerass": "keras", "yachhh": "ya", "gokil": "gila", "ngelihat": "melihat", "ngelawak": "melawak", "iiihhhhh": "ih", "nyolotinnnn": "menyolot", "beratttttttt": "berat", "cantiikkkk": "cantik", "hijaabbb": "hijab", "sekalii": "sekali", "smnagatin": "menyemangati", "pakek": "pakai", "kanapa": "kenapa", "cans": "cantik", "banhet": "banget", "bgttttttt": "banget", "mngasi": "mengasih", "kmbang": "kembang", "pg": "pagi", "pla": "pula", "kw": "kau", "blakon": "berlakon", "bngtzt": "banget", "lucukkkkk": "lucu", "sbelah": "sebelah", "baek": "baik", "ciusss": "serius", "palinf": "paling", "maak": "mak", "ngakaaaakkk": "mengakak", "gokill": "gila", "kocakkkkkk": "kocak", "ngekek": "mengekek", "capee": "cape", "pegangin": "memegang", "nyender": "menyandar", "manjain": "memanjakan", "peleotin": "meleot", "bng": "bang", "termurahhhh": "termurah", "nyatakan": "menyatakan", "graaatiiiis": "gratis", "meriahh": "meriah", "cantikkkm": "cantik", "parahh": "parah", "bingits": "banget", "yuppss": "ya", "nyindir": "menyindir", "onoh": "sono", "anakny": "anaknya", "staf2nya": "staf-stafnya", "begete": "banget", "aduhh": "aduh", "manteppp": "mantap", "kata2nya": "kata-katanya", "ynk": "sayang", "nangis\"": "menangis-nangis", "hati\"": "hati-hati", "jangan\"": "jangan-jangan", "emank": "memang", "cr": "cara", "diemin": "diamkan", "hancurin": "menghancurkan", "diapain": "diapakan", "ms": "masih", "haragain": "menghargai", "cb": "coba", "nyakitin": "menyakiti", "jgn2": "jangan-jangan", "sbaiknya": "sebaiknya", "mngerti": "mengerti", "blajar": "belajar", "mnding": "mending", "buka2": "buka-buka", "nyinyirin": "nyinyir", "nyerang": "menyerang", "trima": "terima", "ngomomg": "mengomong", "mang": "memang", "kbetulan": "kebetulan", "orgtuanya": "orang tuanya", "sbelum": "sebelum", "ngamatin": "mengamati", "ngintip": "mengintip", "stelah": "setelah", "ngintilin": "mengintil", "smnjak": "semenjak", "trbukti": "terbukti", "nyambet": "menyambat", "krmh": "ke rumah", "rmh": "rumah", "bo'ong": "bohong", "eloo": "lo", "apa2nya": "apa-apanya", "goblokkk": "goblok", "sesatt": "sesat", "ibliss": "iblis", "kbiasaan": "kebiasaan", "hdpnya": "hidupnya", "anknya": "anaknya", "tdr": "tidur", "sbuah": "sebuah", "tdnya": "tadinya", "ank2nya": "anak-anaknya", "milih": "memilih", "branjak": "beranjak", "ank2": "anak-anak", "masyarakay": "masyarakat", "menegerti": "mengerti", "nasehatin": "menasihati", "ngaca": "mengaca", "kakkkk": "kak", "slamet": "selamat", "nari": "menari", "ehh": "eh", "mf": "maaf", "ngcpin": "mengucapkan", "woowwww": "wow", "iaa": "iya", "kebagi": "terbagi", "nerusin": "meneruskan", "wihh": "wih", "tetapppp": "tetap", "gantenggg": "ganteng", "jakety": "jaketnya", "nggk": "enggak", "skli": "sekali", "kli": "kali", "pala": "kepala", "pusingggg": "pusing", "pucing": "pusing", "ihhhh": "ih", "maho": "manusia homo", "gantng": "ganteng", "alak": "kepala", "hilangv": "hilang", "bungggg": "bung", "gapap": "enggak apa-apa", "kyak": "kayak", "pling": "paling", "gokiiill": "gila", "busedh": "buset", "gokilnya": "gilanya", "duluuu": "dulu", "keceee": "kece", "nyaa": "nya", "gilaa": "gila", "duuhhh": "duh", "amiiiiinnnnn": "amin", "tullus": "tulus", "tw": "tau", "pluk": "peluk", "durhaka2": "durhaka-durhaka", "kpada": "kepada", "seburuk\"": "seburuk-buruk", "ktaa": "kita", "yup": "ya", "tetaplaah": "tetaplah", "ajja": "saja", "dech": "deh", "oprasi": "operasi", "gya": "gaya", "kndung": "kandung", "u": "lu", "ksenangan": "kesenangan", "ddpt": "didapat", "mmbodohkan": "membodohkan", "orlaen": "orang lain", "malem": "malam", "aplikasiin": "mengaplikasikan", "berangat": "berangkat", "praktekin": "mempraktikkan", "dahh": "deh", "omdo": "omong doang", "kate": "kata", "cabe2": "cabe-cabe", "prjuangan": "perjuangan", "kmewahan": "kemewahan", "stimpal": "setimpal", "ddpat": "didapat", "bc": "baca", "mnilai": "menilai", "usahh": "usah", "segala2nya": "segala-galanya", "sekarng": "sekarang", "mkanya": "makanya", "hay": "hai", "bhagia": "bahagia", "korek2": "korek-korek", "ngebales": "membalas", "sihh": "sih", "sasama": "sesama", "dehh": "deh", "cabe2an": "cabe-cabean", "seklian": "sekalian", "kluar": "keluar", "tuuu": "itu", "ngurusan": "mengurusi", "campurin": "mencampuri", "emnknya": "memangnya", "py": "punya", "unyuu": "unyu", "mrangkul": "merangkul", "wlo": "walau", "manusiy": "manusia", "tx": "tak", "samara": "sakinah mawadah rahman", "teringt": "teringat", "trmsuk": "termasuk", "pilihnmu": "pilihanmu", "swmntara": "sementara", "trmasuk": "termasuk", "liht": "lihat", "suaminy": "suaminya", "jadulnya": "jaman dulunya", "mantaff": "mantap", "ngata2in": "mengatai-ngatai", "bagaimna": "bagaimana", "mulusssss": "mulus", "bgus": "bagus", "gmana": "bagaimana", "kpleset": "kepleset", "yawlah": "ya allah", "kepeleset": "kepleset", "kayanye": "kayanya", "amiinnnn": "amin", "komen2an": "komen-komenan", "wewww": "wow", "dumsss": "dong", "tar": "entar", "perbaikin": "memperbaiki", "permasalhkan": "permasalahkan", "jauhin": "jauhi", "dateng": "datang", "adiik": "adik", "manisss": "manis", "besar\"": "besar-besar", "masing\"": "masing-masing", "ddlm": "di dalam", "kmbali": "kembali", "dkuburnya": "dikuburnya", "fi": "di", "lbih": "lebih", "critamu": "ceritamu", "ja": "saja", "bres": "beres", "cmkn": "camkan", "diurusin": "diurusi", "biarin": "biarkan", "kale": "kali", "donks": "dong", "membanding\"": "membanding-banding", "mengperbolehkan": "memperbolehkan", "krena": "karena", "ngri": "negeri", "dingin\"": "dingin-dingin", "lma": "lama", "dibangga'in": "dibangga-banggakan", "ngebolehin": "membolehkan", "jelek\"in": "menjelek-jelekkan", "stiap": "setiap", "gausa": "enggak usah", "maksain": "memaksakan", "smog": "semoga", "mnerima": "menerima", "baikxa": "baiknya", "aaamin": "amin", "amienn": "amien", "sekangen2": "sekangen-kangen", "amiiiiiiiiiinnnnnnn": "amin", "nolongin": "menolong", "amiiiiiiiiiiin": "amin", "seluru": "seluruh", "amiennn": "amien", "tebir": "ribet", "doangam": "doang", "tebir\"": "ribet-ribet", "amad": "amat", "mkst": "maksud", "wong2": "wong-wong", "rempong": "repot", "ngurusin": "mengurus", "kerjaan": "pekerjaan", "bingittzz": "banget", "bangets": "banget", "ikh": "ih", "dwh": "deh", "nengok": "menengok", "bante": "bantai", "la": "lah", "komenannya": "komentarnya", "hadapin": "menghadapi", "dengarin": "mendengarkan", "ush": "usah", "cock": "cocok", "widihh": "idih", "mantepp": "mantap", "cita2": "cita-cita", "temen-temen": "teman-teman", "stu": "satu", "syratnya": "syaratnya", "pnjang": "panjang", "shat": "sehat", "inginkn": "inginkan", "belanjaa": "belanja", "burem": "buram", "dngan": "dengan", "ngerasain": "merasai", "kancang": "kencang", "tmbh": "tambah", "mansyoallah": "masyaallah", "canttttikkk": "cantik", "subhanalloh": "subhanallah", "lkuin": "lakukan", "langgenh": "langgeng", "foto2nya": "foto-fotonya", "tanggung2": "tanggung-tanggung", "salluutt": "salut", "pkai": "pakai", "pdng": "padang", "warbiyasah": "luar biasa", "keedaannya": "keadaannya", "saya2": "saya-saya", "sekrng": "sekarang", "lagu2nya": "lagu-lagunya", "ttap": "tetap", "spertinya": "sepertinya", "mrah": "marah", "gra2": "gara-gara", "sngaja": "sengaja", "jlek": "jelek", "udah2": "sudah-sudah", "ungkit2": "ungkit-ungkit", "curhat2": "curhat-curhat", "matany": "matanya", "dteng": "datang", "mata'y": "matanya", "baju2": "baju-baju", "buruannn": "buruan", "perubut": "perebut", "sumai": "suami", "urusin": "urus", "apah": "apa", "bagus2": "bagus-bagus", "say": "sayang", "mmbela": "membela", "mslhnya": "masalahnya", "llki": "lelaki", "dluar": "diluar", "sna": "sana", "boleh2": "boleh-boleh", "dya": "dia", "maluuuu": "malu", "beda2": "beda-beda", "nyaranin": "menyarani", "tongolin": "menongol", "cabol": "cabul", "nyet": "monyet", "muna": "munafik", "jing": "anjing", "tot": "entot", "nihh": "nih", "hidiyah": "hidayah", "percya": "percaya", "bgi": "bagi", "tlg": "tolong", "tdi": "tadi", "mgkin": "mungkin", "skaligus": "sekaligus", "cek2": "cek-cek", "spti": "seperti", "mgk": "mungkin", "sapa2": "sapa-sapa", "pnya": "punya", "dng": "dengan", "termurahhh": "termurah", "banyaaaaaak": "banyak", "baik2nya": "baik-baiknya", "tunjukan": "tunjukkan", "anak2mu": "anak-anakmu", "ngasih2": "mengasih-ngasih", "berfikir": "berpikir", "amit2": "amit-amit", "ngajuin": "mengajukan", "terbaeq": "terbaik", "namany": "namanya", "jels": "jelas", "diakibtkn": "diakibatkan", "suk": "suka", "jelek2": "jelek-jelek", "dsrny": "dasarnya", "jkrta": "jakarta", "mantaap": "mantap", "benar2": "benar-benar", "main2": "main-main", "lanjutkaaaaan": "lanjutkan", "hiduppp": "hidup", "ncek": "encek", "majuuu": "maju", "teruuss": "terus", "looohh": "loh", "sebelumbya": "sebelumnya", "tunjukin": "tunjukkan", "tahun\"": "tahun-tahun", "koment": "komentar", "nyampein": "menyampaikan", "hokiii": "hoki", "naksir": "menaksir", "bangt\"": "banget-banget", "nafkahin": "menafkahi", "ngitem": "menghitam", "banyaaaak": "banyak", "sama\"": "sama-sama", "gx": "enggak", "lhat": "lihat", "cantika": "cantik", "cwek": "cewek", "kngen": "kangen", "fillm": "film", "sinet": "sinetron", "jendral": "jenderal", "nanya": "bertanya", "maenya": "mainnya", "cantiks": "cantik", "wktu": "waktu", "pilm": "film", "taon": "tahun", "nd": "indak", "slah": "salah", "cini": "sini", "jaaangan": "jangan", "nambah": "menambah", "naek": "naik", "terimaksih": "terima kasih", "inii": "ini", "kaannnn": "kan", "tidakkkk": "tidak", "kemuh": "kamu", "da": "dah", "sby": "surabaya", "tgl": "tanggal", "pinginnn": "pengin", "buanget": "banget", "ketemuuu": "ketemu", "kebawa": "terbawa", "mimpiii": "mimpi", "wanjerrr": "anjir", "satupunn": "satupun", "motret": "memotret", "lapakk": "lapak", "nyeniii": "menyeni", "bget": "banget", "seeh": "sih", "deuhh": "duh", "b2an": "berduaan", "dihargain": "dihargai", "engga\"": "enggak-enggak", "tahun2": "tahun-tahun", "lebayy": "lebay", "gilsss": "gila", "nganbek": "mengambek", "sukaa": "suka", "enk": "enak", "mendam": "memendam", "manta": "mantan", "relaaaa": "rela", "bgun": "bangun", "tdur": "tidur", "cntk": "cantik", "knl": "kenal", "anugrah": "anugerah", "slm": "salam", "kliatan": "kelihatan", "awoohhh": "allah", "cantiikk": "cantik", "sbbyak": "sebanyak", "apun": "apapun", "nyangkutnya": "menyangkutnya", "ampunnnnn": "ampun", "hy": "hai", "wadaaw": "waduh", "paraah": "parah", "sampw": "sampai", "dijelekin": "dijelekkan", "napa": "kenapa", "azeeeek": "asik", "ntikk": "cantik", "uhh": "uh", "mayan": "lumayan", "mnisss": "manis", "tambahim": "tambahkan", "tambahin": "tambahkan", "bener\"": "benar-benar", "make": "memakai", "canttikkk": "cantik", "baanggeettt": "banget", "ap": "apa", "siiihh": "sih", "krpoooo": "kepo", "pengn": "pengin", "nyusul": "menyusul", "nyok": "yuk", "dekkk": "dek", "ngiri": "mengiri", "nentuin": "menentukan", "mnjmput": "menjemput", "ygs": "sayang", "mblooo": "jomblo", "ber'expresi": "berekspresi", "jdulnya": "judulnya", "ciumman": "ciuman", "aurot": "aurat", "laginy": "laginya", "nyaaaaaaaaaaaaa": "nya", "alkhamdulilah": "alhamdulillah", "ktnya": "katanya", "selaluu": "selalu", "anjirrr": "anjir", "ngeh": "engah", "amen": "amin", "qm": "kamu", "nahhh": "nah", "ngmngnya": "mengomongnya", "sampai2": "sampai-sampai", "fotony": "fotonya", "halllo": "halo", "ngiritya": "mengiritnya", "hematt": "hemat", "ciyeeee": "cie", "yeeee": "ya", "nget": "banget", "kususnya": "khususnya", "krm": "kirim", "sam": "sama", "ajaaahhhhh": "saja", "syediiihhh": "sedih", "malangg": "malang", "belooomm": "belum", "yaakk": "ya", "kebukti": "terbukti", "anak2e": "anak-anaknya", "tuwanya": "tuanya", "comen2nya": "komen-komennya", "pedesssss": "pedas", "nyebut2": "menyebut-nyebut", "plg": "pulang", "smbil": "sambil", "hellooooooo": "halo", "ngrebut": "merebut", "taek": "tahi", "lupain": "melupakan", "bangkret": "bangkrut", "taekk": "tahi", "lahhhhhh": "lah", "kedokk": "kedok", "mati\"an": "mati-matian", "nyimak": "menyimak", "komenx": "komennya", "serem2": "seram-seram", "ngumpet\"": "mengumpet-ngumpet", "blim": "belum", "sekrang": "sekarang", "ngerebut": "merebut", "mnjijikaaaaaan": "menjijikkan", "astgfrlh": "astagfirullah", "sahabatt": "sahabat", "gnyngka": "enggak menyangka", "sendirir": "sendiri", "brengsekk": "brengsek", "ngomentnya": "mengomentarinya", "nusuk": "menusuk", "dpotong": "dipotong", "helo": "halo", "kmrin": "kemarin", "ngmgin": "mengomongkan", "bilangg": "bilang", "gkhianati": "mengkhianati", "sp": "sapa", "pljaran": "pelajaran", "mngikuti": "mengikuti", "klkuan": "kelakuan", "dibikin2": "dibikin-bikin", "kasiannn": "kasihan", "syekaliii": "sekali", "hgga": "hingga", "cieeeee": "cie", "sholihah": "salihah", "adeuh": "aduh", "halahh": "alah", "luu": "lu", "yuuks": "yuk", "yoo": "ya", "ginian": "beginian", "judulnyaa": "judulnya", "ayoookkkkkk": "ayo", "artix": "artinya", "cowo": "cowok", "lt": "lantai", "nama\"": "nama-nama", "kepoin": "kepo", "keluarin": "keluarkan", "gajelas": "enggak jelas", "mbak2": "mbak-mbak", "pedes": "pedas", "komentnya": "komentarnya", "omongin": "omongkan", "ngalirrrrr": "mengalir", "nmbah": "menambah", "terossss": "terus", "ngehujat": "menghujat", "dosaa": "dosa", "ngebela": "membela", "seindonesiyah": "seindonesia", "seh": "sih", "senyumin": "senyumi", "mh": "mah", "ngalir": "mengalir", "ngrusak": "merusak", "px": "punya", "kslhn": "kesalahan", "skrngnya": "sekarangnya", "kalik": "kali", "bilangin": "bilang", "nyambungin": "menyambung", "nggali": "menggali", "cmpur": "campur", "kurng": "kurang", "nangiiiiis": "menangis", "nangiiis": "menangis", "ngomentarin": "mengomentari", "idupnya": "hidupnya", "ortunya": "orang tuanya", "woey": "woy", "bloghhh": "goblok", "ngomongin": "mengomongkan", "doank": "doang", "abal\"": "abal-abal", "dikata2in": "dikata-katai", "ngehina": "menghina", "jngn": "jangan", "mnjawab": "menjawab", "woooiiiiiiii": "woy", "dosaaaaaaaaaa": "dosa", "ingt": "ingat", "karmaaa": "karma", "woooiiiii": "woy", "ngkkkkkk": "mengakak", "makny": "maknya", "wdooohhh": "waduh", "kasihn": "kasihan", "lhirr": "lahir", "hujatann": "hujatan", "palgi": "apalagi", "wihhh": "wih", "kakakkk": "kakak", "bagusss": "bagus", "jt": "juta", "ngets": "banget", "stp": "setiap", "penbelian": "pembelian", "cuaantik": "cantik", "sukakkk": "suka", "m": "sama", "cucokkk": "cocok", "lyt": "lihat", "expresi": "ekspresi", "natap": "menatap", "ccok": "cocok", "helloo": "halo", "dripada": "daripada", "difotoin": "difoto", "psangan": "pasangan", "cakepp": "cakep", "ngiriii": "mengiri", "coxok": "cocok", "buatin": "membuat", "nihhh": "nih", "statusx": "statusnya", "cem": "macam", "yee": "ya", "sunggu": "sungguh", "lagiii": "lagi", "ammiiin": "amin", "haddeh": "aduh", "maaasss": "mas", "segerrrr": "segar", "kemn": "kemana", "suipit": "sipit", "sipiit": "sipit", "kedengeran": "kedengaran", "semangaatt": "semangat", "promil": "program hamil", "lancr": "lancar", "keputihn": "keputihan", "gratisss": "gratis", "kebayang": "terbayang", "mdh\"": "mudah-mudah", "bz": "bisa", "amiinn": "amin", "ammin": "amin", "mendapatkn": "mendapatkan", "lucuuuk": "lucu", "amaattt": "amat", "yaoloh": "yaallah", "ngerayain": "merayakan", "sotoy": "sok tau", "yok": "yuk", "d.bikinin": "dibikin", "mmz": "mas", "bikinin": "membikin", "bapeeeer": "bawa perasaan", "keliatannya": "kelihatannya", "siih": "sih", "diginiin": "dibeginikan", "hadeehh": "aduh", "pu": "punya", "suamiiiii": "suami", "krenn": "keren", "kebykan": "kebanyakan", "tingal": "tinggal", "pernikahn": "pernikahan", "ngtain": "mengatai", "bagusnyeee": "bagusnya", "kemaren": "kemarin", "hmpir": "hampir", "liatnyaaa": "lihatnya", "waw": "wow", "mbuh": "embuh", "waaa": "wah", "berbiiiiiihhh": "barbie", "canteek": "cantik", "teru": "terus", "nyatok": "mencatok", "nyampenya": "sampainya", "goncengin": "boncengkan", "kasi": "kasih", "dianterin": "diantarkan", "gebleg": "geblek", "bosen": "bosan", "anterin": "antarkan", "ngantar": "mengantar", "nganterin": "mengantarkan", "mz": "mas", "nerima": "menerima", "yeuh": "ya", "kmn": "kemana", "dianter": "diantar", "disupirin": "disopiri", "jidad": "jidat", "ciee": "cie", "novembr": "november", "hri": "hari", "bedain": "membedakan", "genduut": "gendut", "ehhh": "eh", "bertatto": "bertato", "brrt": "berarti", "kana": "kanan", "doong": "dong", "sblah": "sebelah", "tauuu": "tau", "tengaj": "tengah", "ps": "pas", "gmpg": "gampang", "engk": "enggak", "apal": "hapal", "dak": "indak", "rupa2nya": "rupa-rupanya", "kembaq": "kembar", "tngh": "tengah", "mknya": "makanya", "prmpuan": "perempuan", "ketuker": "tertukar", "drimnanya": "darimananya", "cntikan": "cantikan", "edaaaan": "edan", "syp": "siapa", "bcnda": "bercanda", "uhhh": "uh", "bazengg": "bajingan", "haiii": "hai", "bosaan": "bosan", "kmana2": "kemana-mana", "komentar2": "komentar-komentar", "kenyataan2": "kenyataan-kenyataan", "silahkan": "silakan", "sukur2": "sukur-sukur", "baek2": "baik-baik", "cma": "cuma", "berbeda2": "berbeda-beda", "dibkatakan": "dikatakan", "doonk": "dong", "pengikuy": "pengikut", "ngmngin": "mengomongkan", "maksd": "maksud", "adkah": "adakah", "menbelah": "membelah", "ayuk": "ayo", "mngira": "mengira", "bertolernsi": "bertoleransi", "slahkan": "salahkan", "tolernsi": "toleransi", "sbnarnya": "sebenarnya", "ingi": "ingin", "oalaaaa": "walah", "menjelekan": "menjelekkan", "hadeuuh": "aduh", "dikemanain": "dikemanakan", "bpknya": "bapaknya", "komen2": "komen-komen", "sebelum2nya": "sebelum-sebelumnya", "sbagai": "sebagai", "sedihh": "sedih", "kangennnn": "kangen", "gndakan": "gandakan", "kangeeeeeeeenya": "kangen", "gaa": "enggak", "ganti'kn": "menggantikan", "tnang": "tenang", "amiiiiiiiiiiiiiiiiin": "amin", "kgeeeen": "kangen", "kangeenn": "kangen", "bangeeeet": "banget", "bsok": "besok", "hbis": "habis", "mti": "mati", "bngetttt": "banget", "kangeen": "kangen", "bgect": "banget", "kebaikan\"y": "kebaikannya", "dssi": "disisi", "kngenn": "kangen", "kangeeeeennn": "kangen", "lagii": "lagi", "dasyat": "dahsyat", "aer": "air", "bgni": "begini", "skalii": "sekali", "gg": "enggak", "dtinggl": "ditinggal", "terbaiiikkkkk": "terbaik", "murahhh": "murah", "pecahhg": "pecah", "kakakgh": "kakak", "terbaeeeek": "terbaik", "prempuan": "perempuan", "manteep": "mantap", "nggeh": "engah", "indonesiaaa": "indonesia", "inia": "ini", "maluu": "malu", "cobaaaa": "coba", "dehhh": "deh", "cepetan": "cepatan", "ntapsss": "mantap", "kamiiii": "kami", "smangat": "semangat", "qmu": "kamu", "sdkit2": "sedikit-sedikit", "penyakitny": "penyakitnya", "ngajarin": "mengajari", "laenny": "lainnya", "amiinnn": "amin", "scpatny": "secepatnya", "sampein": "sampaikan", "tuany": "tuanya", "doa2": "doa-doa", "pjg": "panjang", "adeeek": "adek", "bca": "baca", "netes": "menetes", "mataq": "mataku", "byangin": "membayangkan", "trjadi": "terjadi", "dposisi": "diposisi", "dijwb": "dijawab", "tmn2nya": "teman-temannya", "sayy": "sayang", "salaam": "salam", "ngajar": "mengajar", "fto2": "foto-foto", "laen": "lain", "dallam": "dalam", "tentanh": "tentang", "nangiss": "menangis", "nnton": "menonton", "kejadianny": "kejadiannya", "klg": "keluarga", "sayaang": "sayang", "msg2": "masing-masing", "pgnnya": "penginnya", "meluk": "memeluk", "siiihhh": "sih", "mupeng": "muka pengin", "caya": "percaya", "aduhhhhhhhhh": "aduh", "buseett": "buset", "mblo": "jomblo", "lucuuu": "lucu", "gagahx": "gagahnya", "miriiiip": "mirip", "kaliiii": "kali", "niru": "meniru", "bpkmu": "bapakmu", "sangt": "sangat", "baang": "bang", "gasih": "enggak sih", "yuuu": "yuk", "samaa": "sama", "ngademin": "mengademkan", "aahh": "ah", "makasi": "terima kasih", "looo": "loh", "yokk": "yuk", "gils": "gila", "ono": "sono", "weeeh": "wih", "suksea": "sukses", "temenan": "berteman", "sehh": "sih", "deket": "dekat", "dekt": "dekat", "genggg": "geng", "yawloh": "ya allah", "bhnya": "bahannya", "loo": "lo", "samaaaa": "sama", "abanggku": "abangku", "sayanggg": "sayang", "dsni": "disini", "ciieeeeeee": "cie", "kokohh": "engkoh", "takuttt": "takut", "yuks": "yuk", "pnjual": "penjual", "yh": "ya", "dung": "dong", "latian": "latihan", "aaaaaaakkk": "ah", "mauuuu": "mau", "soga": "moga", "tteh": "teteh", "sintron": "sinetron", "astagaa": "astaga", "ancurrr": "hancur", "dblkang": "dibelakang", "yayang": "sayang", "gokillllll": "gila", "aplks": "aplikasi", "ska": "suka", "ktawa": "ketawa", "baguuus": "bagus", "kaaan": "kan", "ngepoin": "mengepoi", "uuhh": "uh", "beut": "banget", "njirrr": "anjir", "pengeeenn": "pengin", "dongggg": "dong", "hatii": "hati", "besook": "besok", "mauu": "mau", "gtuu": "begitu", "njayyy": "anjay", "maacih": "terima kasih", "tokayyy": "tahi", "to": "tapi", "mksdku": "maksudku", "gituin": "begitukan", "lgsung": "langsung", "cba": "coba", "ilang\"": "hilang-hilang", "giniin": "beginikan", "ksini": "kesini", "kubarikan": "kuberikan", "ayank": "sayang", "anyeng": "anjing", "yankkk": "sayang", "cukurrr": "cukur", "gilaaaaaa": "gila", "yhaaa": "ya", "qaqaaaa": "kakak", "sayangkuuuuu": "sayangku", "kakakuuu": "kakakku", "banx": "bang", "edannn": "edan", "bape": "bawa perasaan", "artis\"": "artis-artis", "ngelahirin": "melahirkan", "paksain": "memaksakan", "biasaa": "biasa", "hub": "hubungan", "hubunggi": "hubungi", "loooh": "loh", "telimakacih": "terima kasih", "nerawang": "menerawang", "cantiiiik": "cantik", "didapet": "didapat", "pede": "percaya diri", "kuningggg": "kuning", "nyolot": "menyolot", "elahh": "alah", "pnting": "penting", "bela2in": "membela-bela", "marah2an": "marah-marahan", "wlaupun": "walaupun", "syantiiiiikkkkkkkkkkkk": "cantik", "bwa\"": "bawa-bawa", "hijabb": "hijab", "kerrennn": "keren", "ngetik": "mengetik", "ntah": "entah", "pan": "kapan", "mateeeek": "mati", "geetoo": "begitu", "sewain": "menyewa", "kokkk": "kok", "muluuu": "melulu", "yaaang": "yang", "iyaa": "iya", "gituu": "begitu", "yappp": "ya", "uwahhhhh": "wah", "bnr2": "benar-benar", "psgn": "pasangan", "ujg": "ujung", "rmbut": "rambut", "smp": "sampai", "kki": "kaki", "hayukk": "hayo", "ntab": "mantab", "atuhhh": "atuh", "yukss": "yuk", "lihaatttt": "lihat", "niiiihh": "nih", "liattt": "lihat", "cuyyy": "cuy", "gapernah": "enggak pernah", "liatiii": "lihat", "jombsss": "jomblo", "emesshhh": "gemas", "gemeessss": "gemas", "poollll": "pol", "gemesyak": "gemas ya", "bngttt": "banget", "ngegmesin": "menggemaskan", "hndak": "hendak", "hndk": "hendak", "iiih": "ih", "lutunaa": "lucunya", "abiz": "habis", "gemes": "gemas", "ngareppppp": "mengharap", "aamiinin": "amin", "imuutt": "imut", "abizzz": "habis", "tay": "tahi", "toketnya": "teteknya", "tumpe2": "tumpah-tumpah", "lenje": "lenjeh", "tuuuuh": "tuh", "cantinya": "cantiknya", "mbak\"": "mbak-mbak", "ngatain": "mengatai", "mbaknyaahh": "mbaknya", "pyudara": "payudara", "ditutupin": "ditutupi", "bgitu": "begitu", "ngomongnya": "mengomongnya", "sich": "sih", "cantiik2": "cantik-cantik", "hati.a": "hatinya", "mk.a": "makanya", "mlt.a": "mulutnya", "nyerocooooos": "mencerocos", "kyk.a": "kayaknya", "sklh": "sekolah", "pljrn": "pelajaran", "srng": "sering", "ngilang": "menghilang", "dikatain": "dikatai", "maluuu": "malu", "ngalahin": "mengalahkan", "brisi": "berisi", "pkoknya": "pokoknya", "canti": "cantik", "astagfirullh": "astagfirullah", "nutupin": "menutupi", "ori": "orisinal", "kakakaaaaaaaaaaaaaa": "kakak", "cantikkkkkkkk": "cantik", "batwc": "banget", "kamyuhhh": "kamu", "dbwh": "dibawah", "senenggg": "senang", "mao": "mau", "murhan": "murahan", "prwtn": "perawatan", "wjh": "wajah", "bgettt": "banget", "pakke": "pakai", "ktergantungan": "ketergantungan", "prwatn": "perawatan", "ngebuktiin": "membuktikan", "hslnya": "hasilnya", "awal2": "awal-awal", "setabil": "stabil", "maksih": "terima kasih", "gratiss": "gratis", "bbrpa": "beberapa", "gara\"": "gara-gara", "benar\"": "benar-benar", "ajaibb": "ajaib", "pokonya": "pokoknya", "setujuuu": "setuju", "digituin": "dibegitukan", "sekolahin": "sekolahkan", "ajarin": "ajarkan", "sendirii": "sendiri", "mrnghadapi": "menghadapi", "naka": "nakal", "besar2in": "besar-besarkan", "bgmana": "bagaimana", "s*inetr*n": "sinetron", "mrusak": "merusak", "mndpt": "mendapat", "skt": "sakit", "sebnyak": "sebanyak", "terburu2": "terburu-buru", "baik2": "baik-baik", "ngumpat2": "mengumpat-ngumpat", "nantangin": "menantang", "ngegampar": "menggampar", "ngelaporin": "melaporkan", "salahin": "salahkan", "gabisa": "enggak bisa", "konsekwensi": "konsekuensi", "muliany": "mulianya", "mungki": "mungkin", "t4": "tempat", "bacahalah": "bacalah", "dikerasin": "dikerasi", "gkkan": "enggak akan", "ngulangin": "mengulangi", "bedaa": "beda", "pnyaa": "punya", "bol3h": "boleh", "ngadu": "mengadu", "ketauan": "ketahuan", "dimarahin": "dimarahi", "nyelesain": "menyelesaikan", "troma": "trauma", "kerasin": "kerasi", "cewer": "jewer", "bilanh": "bilang", "jwban": "jawaban", "nuduh": "menuduh", "ngatur": "mengatur", "rajin2": "rajin-rajin", "bacaaaa": "baca", "kekk": "kayak", "jungkiir": "jungkir", "gunaa": "guna", "bales2in": "balas-membalas", "orangg": "orang", "tauuuu": "tau", "byak": "banyak", "kepad": "kepada", "bantuanny": "bantuannya", "ttinggi": "tinggi", "perna": "pernah", "grgr": "gara-gara", "jari2": "jari-jari", "pukulin": "pukuli", "bengkak2": "bengkak-bengkak", "komentarin": "mengomentari", "bijak2": "bijak-bijak", "lama2": "lama-lama", "menurur": "menurut", "tipe2": "tipe-tipe", "diliat": "dilihat", "liatlah": "lihatlah", "teguran2": "teguran-teguran", "gr2": "gara-gara", "skolah": "sekolah", "pdahal": "padahal", "klas": "kelas", "tkut": "takut", "grunya": "gurunya", "pndah": "pindah", "liat2": "lihat-lihat", "masukin": "memasuki", "dilindungin": "dilindungi", "kasiaannn": "kasihan", "miring2": "miring-miring", "tinggalin": "tinggalkan", "kaitannha": "kaitannya", "sud": "sudah", "pinter": "pintar", "rana": "ranah", "bhkn": "bahkan", "justeru": "justru", "ngatai": "mengatai", "kln": "kalian", "sukak": "suka", "termuraaahhhh": "termurah", "brkunjun": "berkunjung", "suarranya": "suaranya", "bner\"": "benar-benar", "iu": "itu", "bgussss": "bagus", "kmaren": "kemarin", "ambilin": "mengambil", "biasaaaa": "biasa", "luuuaar": "luar", "redho": "ridho", "rezki": "rezeki", "kepd": "kepada", "luaarrr": "luar", "biasaaa": "biasa", "jadilh": "jadilah", "bnr": "benar", "terhaaru": "terharu", "orrunya": "orang tuanya", "flm": "film", "kalok": "kalo", "adek2": "adek-adek", "loncatt": "loncat", "hampasss": "hempas", "tehh": "teh", "bunga2": "bunga-bunga", "apaaan": "apaan", "ampuuunn": "ampun", "daaah": "dah", "bangetttttt": "banget", "ituh": "itu", "cantiiikk": "cantik", "suaranyaaaaaa": "suaranya", "idoolaaa": "idola", "pulaa": "pula", "penyanyi2": "penyanyi-penyanyi", "wiih": "wih", "apaa": "apa", "miriip": "mirip", "allaahh": "allah", "ajahhh": "saja", "trims": "terima kasih", "dehat": "sehat", "salammmm": "salam", "sihat": "sehat", "buruann": "buruan", "rekomendain": "rekomendasikan", "trussss": "terus", "bangt": "banget", "cepet": "cepat", "pesen": "pesan", "boong": "bohong", "prawatan": "perawatan", "mkin": "makin", "dimao": "dimau", "dijadiin": "dijadikan", "silahkn": "silakan", "kasiatnya": "khasiatnya", "minyak\"an": "minyak-minyakan", "suksess": "sukses", "prodak": "produk", "trasa": "terasa", "minpi": "mimpi", "bner2": "benar-benar", "mutihin": "memutihkan", "smster": "semester", "kmren": "kemarin", "ucull": "lucu", "ngomong\"": "ngomong-ngomong", "udehh": "sudah", "syngnya": "sayangnya", "dipermainin": "dipermainkan", "kocakkk": "kocak", "dasarrr": "dasar", "waaaaw": "wow", "cannnnntikkkkkkk": "cantik", "mukax": "mukanya", "cantikx": "cantiknya", "fatanik": "fanatik", "protestin": "protestan", "kakau": "kalau", "marh": "marah", "emngnya": "memangnya", "cantiknyaa": "cantiknya", "darpada": "daripada", "cooooooocok": "cocok", "pkk": "pakai", "sling": "saling", "agama'ny": "agamanya", "bilng": "bilang", "prcy": "percaya", "lurusin": "meluruskan", "biri": "biru", "luco": "lucu", "ultha": "ultah", "batikk": "batik", "arab2": "arab-arab", "luchu": "lucu", "baguss": "bagus", "sedihhh": "sedih", "syang": "sayang", "ngasi": "mengasih", "naaaak": "nak", "astafirullah": "astagfirullah", "peragain": "peragakan", "shayyy": "say", "anjaayy": "anjay", "shay": "say", "ngilerrrr": "mengiler", "woyyy": "woy", "shaaaay": "say", "hempaass": "hempas", "sayyy": "say", "shayy": "say", "syantiek": "cantik", "njay": "anjay", "hehh": "eh", "kam": "kamu", "ingeet": "ingat", "sblh": "sebelah", "nawarin": "menawari", "amiiiiin": "amin", "jauhhh": "jauh", "waaahhh": "wah", "ajakin": "mengajak", "mntn": "mantan", "ade": "adek", "luhhh": "lu", "yooookkk": "yuk", "jrnk": "jarang", "jelas\"": "jelas-jelas", "kayax": "kayak", "dblkng": "dibelakang", "anjirr": "anjir", "mirippp": "mirip", "sangattt": "sangat", "iyap": "iya", "mayann": "lumayan", "autuhh": "tau tuh", "atutt": "takut", "ketara": "tertara", "giniii": "begini", "mbaaass": "mbak", "bund": "bunda", "bangeeettt": "banget", "almh": "almarhumah", "temenkuuh": "temanku", "suami'y": "suaminya", "bngeeeetttt": "banget", "entr": "entar", "kyax": "kayak", "cuantikkk": "cantik", "mksdnya": "maksudnya", "kaaaaaaaaaakkkkkkkkkkk": "kak", "kucing'y": "kucingnya", "nama'in": "namai", "panggilan'ya": "panggilannya", "panggilan'y": "panggilannya", "z": "saja", "kalw": "kalau", "tips2": "tips-tips", "alu": "aku", "ngerrrriiii": "ngeri", "ngriii": "mengiri", "amatt": "amat", "smpai": "sampai", "kox": "kok", "tmbah": "tambah", "cntikk": "cantik", "pkee": "pakai", "smkin": "semakin", "amiiiiinnn": "amin", "bgmna": "bagaimana", "ssorg": "seseorang", "hdpn": "hadapan", "mnambh": "menambah", "ktaatan": "ketaatan", "stju": "setuju", "cantikkkkkk": "cantik", "dilepas2": "dilepas-lepas", "tehhh": "teh", "istiqomahhh": "istikamah", "tehhhh": "teh", "prodaknya": "produknya", "imutt": "imut", "amu": "kamu", "nahh": "nah", "mauuuuuuu": "mau", "jilbb": "jilbab", "ndut": "gendut", "ngecek": "mengecek", "psti": "pasti", "penasarn": "penasaran", "cuyy": "cuy", "astauviruloh": "astagfirullah", "paaak": "pak", "diajarin": "diajari", "gawl": "gaul", "kecr": "kece", "iih": "ih", "laaaah": "lah", "aned": "banget", "gingsungny": "gingsulnya", "cariin": "carikan", "kenceng": "kencang", "cuanti": "cantik", "mbanya": "mbaknya", "knlan": "kenalan", "mbaakk": "mbak", "nyusahhin": "menyusahkan", "byaran": "bayaran", "wlawpun": "walaupun", "sedangkn": "sedangkan", "tnpa": "tanpa", "bnggain": "banggakan", "nyusahin": "menyusahkan", "malu2in": "malu-malui", "dibanggain": "dibanggakan", "wlau": "walau", "komenan": "komentar", "bangga\"in": "membangga-banggakan", "krya": "karya", "bilang\"": "bilang-bilang", "tinggall": "tinggal", "dmna": "dimana", "dripda": "daripada", "bangga2in": "membangga-banggakan", "dibnggain": "dibanggakan", "karya2": "karya-karya", "drimana": "darimana", "ngepake'": "memakai", "gksalah": "enggak salah", "nuntut": "menuntut", "ngebiarin": "membiarkan", "bhas": "bahas", "lagu'y": "lagunya", "jadiin": "jadikan", "diem": "diam", "nyampah": "menyampah", "mba\"": "mbak-mbak", "bukan'a": "bukannya", "tappi": "tapi", "kalli": "kali", "pacara": "pacaran", "ngerebut\"": "merebut-rebut", "lluu": "lu", "mbaa": "mbak", "ngerussak": "merusak", "bela\"in": "membela-bela", "bellain": "membela", "kallo": "kalo", "coment": "komentar", "dewassa": "dewasa", "comnet": "komen", "pikir\"": "pikir-pikir", "kaya'a": "kayanya", "kuyyy": "yuk", "kitaa": "kita", "sgtu'y": "sebegitunya", "ngebelain": "membela", "tpii": "tapi", "karya\"": "karya-karya", "do": "di", "cali": "caci", "penghrgaan": "penghargaan", "ajjh": "saja", "lbihh": "lebih", "baikk": "baik", "drii": "dari", "artist\"": "artis-artis", "maki\"": "maki-maki", "sendirinnya": "sendirinya", "stidaknya": "setidaknya", "may": "mau", "trbaikk": "terbaik", "punn": "pun", "mengikutii": "mengikuti", "atassan": "atasan", "klauu": "kalau", "jngnn": "jangan", "so": "sok", "sendirinyya": "sendirinya", "frestasiii": "prestasi", "mst": "mesti", "add": "ada", "sibukx": "sibuknya", "in": "ini", "katanyaa": "katanya", "lawaaaas": "lawas", "kocaaaak": "kocak", "syng": "sayang", "adenya": "adeknya", "pkoknyaa": "pokoknya", "sichhhhhhhhhh": "sih", "nampak": "menampak", "tetehh": "teteh", "wooowww": "wow", "cobain": "mencoba", "muga": "moga", "kenalin": "memperkenalkan", "janka": "jangka", "wawwwww": "wow", "waaaww": "wow", "waaawwww": "wow", "mantapp": "mantap", "cobaaa": "coba", "kageet": "kaget", "bangettttttt": "banget", "oplas": "operasi plastik", "nambahh": "bertambah", "paripurnahh": "paripurna", "wanitah": "wanita", "indonesiaah": "indonesia", "cumn": "cuman", "gegra": "gegara", "brita": "berita", "fotbar": "foto bareng", "wajarrr": "wajar", "wk": "waktu", "bingiittt": "banget", "nampar": "menampar", "tukan": "tukang", "jatoh": "jatuh", "muka\"": "muka-muka", "cantik2nya": "cantik-cantiknya", "sdgkn": "sedangkan", "apa\"": "apa-apa", "mancungin": "memancungkan", "idung": "hidung", "kusem": "kusam", "drpada": "daripada", "kmar": "kamar", "diemm": "diam", "tnggu": "tunggu", "ketawain": "mentertawakan", "yaelahhh": "alah", "sikap\"": "sikap-sikap", "bales": "balas", "ampuun": "ampun", "ngejatuhin": "manjatuhkan", "ngelakuin": "melakukan", "gedeg": "gedek", "muka2": "muka-muka", "eey": "eh", "kslahan": "kesalahan", "perhatiam": "perhatian", "dibiarin": "dibiarkan", "sbnrny": "sebenarnya", "trjd": "terjadi", "mkir": "memikir", "apalai": "apalagi", "bkal": "bakal", "saat2": "saat-saat", "nyadar": "menyadar", "bka": "buka", "berani2": "berani-berani", "lucuuuuu": "lucu", "pipi'y": "pipinya", "gemmeeezzz": "gemas", "sukam": "suka", "menghilangkn": "menghilangkan", "mnjdkan": "menjadikan", "mudah2n": "mudah-mudahan", "soalx": "soalnya", "keche": "kece", "badaii": "badai", "btuh": "butuh", "ktanya": "katanya", "bebrp": "beberapa", "sblum": "sebelum", "santab": "santap", "salehah": "salihah", "mikirin": "memikirkan", "subhannallah": "subhanallah", "konsul": "konsultasi", "pengalam": "pengalaman", "poko": "pokok", "trun": "turun", "kuninggg": "kuning", "pinkkkk": "pink", "warnany": "warnanya", "seger": "segar", "senangat": "semangat", "msknya": "masaknya", "kerennnn": "keren", "ciptain": "menciptakan", "bagussss": "bagus", "sayangkuh": "sayangku", "gans": "ganteng", "kren": "keren", "babget": "banget", "dihh": "dih", "jadu": "jadi", "ksyngn": "kesayangan", "haru": "harus", "ngingetin": "mengingatkan", "narik": "menarik", "orang\"": "orang-orang", "gokilll": "gila", "astagaaaa": "astaga", "hancuuuuuy": "hancur", "mattihhh": "mati", "beudd": "banget", "termiribkan": "termiripkan", "dngerin": "mendengarkan", "brrti": "berarti", "wenakk": "enak", "nganteng": "ganteng", "abanggg": "abang", "seksii": "seksi", "gtuh": "begitu", "jadii": "jadi", "emeezzz": "gemas", "amohhh": "kamu", "maau": "mau", "eyke": "aku", "gilakkkkk": "gila", "awk": "awak", "duhhhhh": "duh", "wowww": "wow", "liaatt": "lihat", "iniiiii": "ini", "aloh": "allah", "ngettt": "banget", "maso": "masa", "brubah": "berubah", "pelastik": "plastik", "keak": "kayak", "dengerin": "mendengarkan", "beud": "banget", "pahamin": "memahami", "takuuuuut": "takut", "ahahahanjirrr": "anjir", "jogetin": "berjoget", "shaay": "say", "boker": "berak", "pacarnyaa": "pacarnya", "siapaa": "siapa", "matii": "mati", "ketawaa": "ketawa", "wanjiir": "anjir", "minta": "meminta", "bnran": "benaran", "sgla": "segala", "uitnya": "duitnya", "beliiin": "belikan", "yaaah": "ya", "sekaeang": "sekarang", "tanyain": "tanyakan", "jatuhnyaa": "jatuhnya", "banyakin": "membanyaki", "tipu2nya": "tipu-tipunya", "doaang": "doang", "gituan": "begituan", "kaj": "kak", "wahhhhhh": "wah", "iiiiiiiii": "ih", "kaliiiiiii": "kali", "melebihin": "melebihi", "gakpapa": "enggak apa-apa", "kalogak": "kalo enggak", "sm2": "sama-sama", "ndukung": "mendukung", "kaaaaa": "kak", "aamiinnn": "amin", "skluarga": "sekeluarga", "suaminga": "suaminya", "berhijap": "berhijab", "smgaa": "semoga", "istiqamah": "istikamah", "cuaaantikk": "cantik", "muraaaaah": "murah", "baraang": "barang", "aamiiin": "amin", "bets": "banget", "alhamdulillaaaaah": "alhamdulillah", "sembuhin": "menyembuhkan", "pengrusak": "perusak", "ngoceh": "mengoceh", "aduhhh": "aduh", "hpus": "hapus", "pjang": "pajang", "bdampingan": "berdampingan", "uih": "uh", "serem\"": "seram-seram", "happi\"": "hepi-hepi", "bgtuu": "begitu", "mff": "maaf", "ktwa": "ketawa", "trlalu": "terlalu", "jjur": "jujur", "ngmg": "mengomong", "semogaa": "semoga", "kepelesett": "terpeleset", "geeelaakk": "gila", "bolang": "bocah petualang", "pkek": "pakai", "lagihh": "lagi", "hedeuuhh": "aduh", "stress": "stres", "saltummmm": "salah kostum", "mboizzz": "bois", "suda": "sudah", "kayaq": "kayak", "ema": "emak", "dirumahny": "dirumahnya", "yuh": "tuh", "rmbt": "rambut", "apus": "hapus", "siapayahh": "siapa ya", "kostom": "kostum", "gnii": "begini", "atuu": "satu", "pipinyaaa": "pipinya", "bungkuuuuuuuuusss": "bungkus", "iiiihhhhhj": "ih", "emeessss": "gemas", "pipine": "pipinya", "glembeeeer": "gelambir", "lucunaaa": "lucunya", "cubanget": "lucu banget", "pengenn": "pengin", "unyuuk": "unyu", "dedek": "adek", "lucunyah": "lucunya", "mantaaap": "mantap", "keduaa": "kedua", "trsrh": "terserah", "ksih": "kasih", "seterong": "setrong", "teyuss": "terus", "dimasukin": "dimasuki", "jelekin": "menjelekkan", "jahatin": "menjahati", "tai": "tahi", "gtw": "enggak tau", "qo": "kok", "nulis": "menulis", "nyebutin": "menyebutkan", "ngertiin": "mengerti", "rasaiin": "rasai", "gakusah": "enggak usah", "disni": "disini", "sabarrr": "sabar", "semangaatttt": "semangat", "nyalahin": "menyalahkan", "bedainnya": "membedakannya", "krus": "kurus", "elaaaahhhh": "alah", "cogan": "cowok ganteng", "tap8": "tapi", "pandanginya": "memandanginya", "kenaliiin": "kenalkan", "banyak2": "banyak-banyak", "yo": "ya", "suko": "suka", "niih": "nih", "sihhhh": "sih", "sebaik\"": "sebaik-baik", "segtu": "sebegitu", "ngebenci": "membenci", "bakar2in": "bakar-bakari", "padhe": "pakde", "yya": "ya", "dosa2mu": "dosa-dosamu", "maafin": "memaafi", "dalem2": "dalam-dalam", "kgak": "kagak", "bahas2": "bahas-bahas", "woles": "selow", "hianatin": "khianati", "halangin": "halangi", "iyuh": "ih", "iyuuuhhhh": "ih", "gaje": "enggak jelas", "gabakal": "enggak bakal", "adapin": "menghadapi", "ngehakimi": "menghakimi", "ngejuangin": "memperjuangkan", "dipojokin": "dipojokkan", "bacotin": "bacoti", "berati": "berarti", "kebangetak": "kebangetan", "tangkep": "tangkap", "bt": "buat", "sadis2": "sadis-sadis", "maksir": "menaksir", "hebatt": "hebat", "pndang": "pandang", "pmbelajaran": "pembelajaran", "ngejelek": "menjelekkan", "maslah": "masalah", "mmperbaiki": "memperbaiki", "nahan": "menahan", "kafeh": "kafe", "remang2": "remang-remang", "mlh": "malah", "tinggl": "tinggal", "dirmh": "dirumah", "iyuuhhh": "ih", "ktika": "ketika", "dipuja2": "dipuja-puja", "ngrasain": "merasai", "ityl": "itil", "hbs": "habis", "wenak": "enak", "ngarang": "mengarang", "kelas'a": "kelasnya", "hadeeehhh": "aduh", "kecil\"": "kecil-kecil", "gtu\"an": "begitu-gituan", "gmanapun": "bagaimanapun", "kr": "kira", "nenek2": "nenek-nenek", "taux": "taunya", "urat2nya": "urat-uratnya", "ketarik": "tertarik", "ngorbanin": "mengorbankan", "eksprsi": "ekspresi", "gitulah": "begitulah", "foto\"": "foto-foto", "mamas": "mas", "bandingin": "bandingkan", "tua2": "tua-tua", "terus2an": "terus-terusan", "waduhh": "waduh", "bawa\"": "bawa-bawa", "badingkan": "membandingkan", "ngurus": "mengurus", "lanjutin": "lanjutkan", "ributin": "meributkan", "brantem": "berantem", "belain": "membela", "lipetan": "lipatan", "nenek\"": "nenek-nenek", "bneraan": "benaran", "kageeet": "kaget", "sangaaattt": "sangat", "rmbuut": "rambut", "nt": "nanti", "walik": "balik", "oma2": "oma-oma", "kellen": "kalian", "nyerocos": "mencerocos", "ibuk2": "ibu-ibu", "ngomenin": "mengomentari", "iseng2": "iseng-iseng", "niyyyyhhhh": "nih", "sexi2": "seksi-seksi", "blik": "balik", "kenape": "kenapa", "ladenin": "meladeni", "skit": "sakit", "hat": "hati", "jilbabnye": "jilbabnya", "ledkin": "meledeki", "digabtikan": "digantikan", "ngumung": "mengomong", "ilang-in": "hilangkan", "prinadi": "pribadi", "luh": "lu", "yep": "ya", "helooooo": "halo", "palingann": "palingan", "berjinah": "berzina", "munaafikk": "munafik", "urusn": "urusan", "saudra": "saudara", "trganggu": "terganggu", "bhijab": "berhijab", "lepasin": "melepas", "waduhhh": "waduh", "ursan": "urusan", "sudab": "sudah", "dwsa": "dewasa", "rpt2": "repot-repot", "ngaji": "mengaji", "nmr": "nomor", "romantiz": "romantis", "murmer": "murah meriah", "pensaran": "penasaran", "yuukk": "yuk", "kitaaa": "kita", "lainnyaa": "lainnya", "perbulannn": "perbulan", "penyebab2": "penyebab-penyebab", "nyupir": "menyopir", "steess": "stres", "kwatir": "kawatir", "duetin": "duet", "mudaa": "muda", "silsturahmi": "silaturahmi", "dijkt": "dijakarta", "cums": "cuma", "undangan2": "undangan-undangan", "past": "pasti", "ayukk": "yuk", "ngucapin": "mengucapkan", "filmny": "filmnya", "ntn": "menonton", "nobar": "menonton bareng", "kemariin": "kemarin", "knpp": "kenapa", "lggi": "lagi", "gaprgi": "enggak pergi", "aaahh": "ah", "pa2": "apa-apa", "dtunggu": "ditunggu", "masa2": "masa-masa", "kepikir": "terpikir", "seruuu": "seru", "emeshh": "gemas", "kaya": "kayak", "astagaaaaaa": "astaga", "pacarn": "pacaran", "ktg": "kita", "pacaranya": "pacarannya", "mbaak": "mbak", "ngalah": "mengalah", "de": "deh", "pacarane": "pacarannya", "rusakkk": "rusak", "yaaampunnn": "ya ampun", "tuwek": "tua", "mukae": "mukanya", "ultahe": "ultahnya", "kliatanya": "kelihatannya", "pcaranya": "pacarannya", "bermaksut": "bermaksud", "tampialannya": "tampilannya", "nikaah": "nikah", "anak\"": "anak-anak", "pankapan": "kapan-kapan", "ngumbar": "mengumbar", "dhpusin": "dihapuskan", "istiqomah": "istikamah", "mnutup": "menutup", "bngatt": "banget", "dii": "di", "ooh": "oh", "hapusin": "hapuskan", "lohhh": "loh", "ngeuh": "engah", "sneng": "senang", "mutusin": "memutuskan", "ngapus": "menghapus", "awl": "awal", "hrus": "harus", "bngkit": "bangkit", "mncoba": "mencoba", "wahhh": "wah", "mutuain": "memutuskan", "geuliiiiiiis": "geulis", "euyyyyy": "euy", "uuh": "uh", "ngirim": "mengirim", "ngidam": "mengidam", "waah": "wah", "sampis": "sampah habis", "maapin": "maafi", "ngebala": "bala", "mampuussss": "mampus", "diihhhh": "dih", "hrusnya": "harusnya", "sabal": "sabar", "udahlah": "sudahlah", "kauuuu": "kau", "sabaro": "sabar", "eehhh": "eh", "iiii": "ih", "kreative": "kreatif", "bangrt": "banget", "azekkkk": "asik", "kuaaatt": "kuat", "bhet": "banget", "inggrisny": "inggrisnya", "dsna": "disana", "iyaap": "iya", "mksih": "terima kasih", "canyk": "cantik", "prodac": "produk", "nyesell": "menyesal", "ciyusss": "serius", "cyin": "cinta", "bangetttttttt": "banget", "sexi": "seksi", "bundaaaa": "bunda", "bangget": "banget", "bettt": "banget", "cantiiillx": "cantik", "brapa": "berapa", "alaami": "alami", "sabaarrr": "sabar", "cantiiiikk": "cantik", "bangeeeeetttt": "banget", "cannntikk": "cantik", "banggett": "banget", "thnan": "tahunan", "cuantik": "cantik", "segerr": "segar", "wouww": "wow", "cantiiikkk": "cantik", "pen": "pengin", "catokin": "catok", "comment2nya": "komentar-komentarnya", "knsul": "konsultasi", "brhasil": "berhasil", "hilangin": "menghilangkan", "kontak2": "kontak-kontak", "dptnya": "dapatnya", "applikasi": "aplikasi", "kereeeeen": "keren", "bngeet": "banget", "gemesinnnnn": "menggemaskan", "ongkiir": "ongkir", "ajaibbbbb": "ajaib", "saayyyyy": "say", "lohhhh": "loh", "ganteung": "ganteng", "bhgia": "bahagia", "iiiiiiiiii": "ih", "lucuukk": "lucu", "keciil": "kecil", "yakinn": "yakin", "pentass": "pentas", "ngarep": "mengharap", "lucu2": "lucu-lucu", "testinya": "testimoninya", "mengirusi": "mengurusi", "khdpn": "kehidupan", "ngelawan": "melawan", "betuuuuulllllllllllllllll": "betul", "bisax": "bisanya", "klmbutan": "kelembutan", "seburuk2": "seburuk-buruk", "diingetin": "diingatkan", "ngeriiiiii": "ngeri", "nich": "nih", "akhlah": "akhlak", "elo2": "lo-lo", "mandiin": "memandikan", "nguburin": "menguburkan", "ngebedain": "membedakan", "urusanny": "urusannya", "tuhanny": "tuhannya", "brtngkar": "bertengkar", "pndapat": "pendapat", "looh": "loh", "sharusnya": "seharusnya", "mngkannya": "mangkanya", "nyia2in": "menyia-nyiakan", "brarti": "berarti", "mndpat": "mendapat", "kpuasan": "kepuasan", "sgampang": "segampang", "mlepaskan": "melepaskan", "trll": "terlalu", "sndr": "sendiri", "sellow": "selow", "sbbrnya": "sebenarnya", "kasar2": "kasar-kasar", "semangattt": "semangat", "testi": "testimoni", "banyaak": "banyak", "bujukk": "bujuk", "d.apa": "diapa", "apain": "apai", "benetaran": "benaran", "bneran": "benaran", "si": "sih", "kraya": "karya", "brg": "bareng", "ngebantu": "membantu", "janjutin": "lanjutkan", "kepppoin": "kepoi", "recommendasi": "rekomendasi", "btulan": "betulan", "kha": "kah", "s": "sih", "tutupin": "menutupi", "wik": "wig", "ithu": "itu", "botakin": "botaki", "botakkkk": "botak", "bnrn": "benaran", "week": "wig", "gawul": "gaul", "modes": "modis", "brayyy": "bray", "kcantikan": "kecantikan", "bronis": "brondong manis", "gapunya": "enggak punya", "bkln": "bakalan", "kecantikanga": "kecantikannya", "hiii": "hai", "piiiilih": "pilih", "deeh": "deh", "zama": "zaman", "gnti": "ganti", "nmnya": "namanya", "tapo": "tapi", "gkk": "enggak", "petinggi2": "petinggi-petinggi", "indefendet": "independen", "tempat2": "tempat-tempat", "taik": "tahi", "yuuuukkk": "yuk", "dituutup": "ditutup", "ngayal": "mengkhayal", "smvk": "sempak", "knyataan": "kenyataan", "njrittt": "anjir", "taz": "atas", "klhran": "kelahiran", "pertma": "pertama", "kelahuran": "kelahiran", "pertamany": "pertamanya", "mamax": "mamanya", "ankx": "anaknya", "cantip": "cantik", "klahiran": "kelahiran", "slmat": "selamat", "mmbanggakan": "membanggakan", "ortuny": "orang tuanya", "selmat": "selamat", "nyah": "nya", "prtamax": "pertamanya", "moqa": "moga", "yq": "yang", "brbakti": "berbakti", "gewlaaaaa": "gila", "basodaraa": "bersaudara", "hargs": "harga", "perhatiin": "perhatikan", "bantuim": "bantu", "pelwak": "pelawak", "suces": "sukses", "sucses": "sukses", "garemin": "garami", "gapake": "enggak pakai", "ngiringan": "mengiringi", "karir.a": "karirnya", "aamin": "amin", "teruz": "terus", "kemarn": "kemarin", "recomendasinya": "rekomendasinya", "promoo": "promo", "ongkirr": "ongkir", "mskasih": "terima kasih", "wowwwww": "wow", "seeeemua": "semua", "batesin": "batasi", "bijin": "bikin", "ngeidealin": "mengidealkan", "bhn": "bahan", "gasalah": "enggak salah", "brsma": "bersama", "le": "boleh", "ugha": "juga", "semuaaa": "semua", "heeee": "hai", "rame2": "rame-rame", "yam": "ya", "kartunin": "kartun", "kyaknya": "kayaknya", "abez": "habis", "baaaaaang": "bang", "ganteeeeng": "ganteng", "entahhh": "entah", "gimanaaaa": "bagaimana", "duuhh": "duh", "gantengnyaaaa": "gantengnya", "kpn2": "kapan-kapan", "pans": "fan", "kuliyh": "kuliah", "gantengkuuuuuu": "gantengku", "atassss": "atas", "orgny": "orangnya", "owalaah": "walah", "aslinyo": "aslinya", "alissnyaa": "alisnya", "masaaaa": "masa", "komuk": "muka", "kalahnye": "kalahnya", "iyaaaa": "iya", "fakultass": "fakultas", "smrg": "semarang", "unyu2": "unyu-unyu", "yyaaa": "ya", "insonniaaa": "insomnia", "adekx": "adeknya", "gksi": "enggak sih", "greged": "greget", "alamiii": "alami", "ktemu": "ketemu", "ditayangin": "ditayangkan", "ms\"": "masa-masa", "meranin": "memerankan", "nontonny": "menontonnya", "rekomendasiiin": "merekomendasikan", "sista2": "sista-sista", "agan2": "agan-agan", "ngg": "enggak", "nanget": "banget", "ngeditnya": "mengeditnya", "himana": "bagaimana", "kekgini": "kayak begini", "subhanaallahh": "subhanallah", "sangat2": "sangat-sangat", "terharuuuu": "terharu", "kata\"": "kata-kata", "subhanallahhh": "subhanallah", "tlsn": "tulisan", "nagiss": "menangis", "aneut": "banget", "trrharu": "terharu", "subhnallah": "subhanallah", "allahh": "allah", "suamiii": "suami", "dcintai": "dicintai", "klrga": "keluarga", "liat'y": "lihatnya", "lggeng": "langgeng", "romantisny": "romatisnya", "duhhh": "duh", "pcar": "pacar", "kakanga": "kakaknya", "slw": "selow", "cpa": "siapa", "akuu": "aku", "aelah": "alah", "orabg": "orang", "mukanyaa": "mukanya", "kakakny": "kakaknya", "woooyy": "woy", "ngelucu": "melucu", "bukak": "buka", "mato": "mata", "kareno": "karena", "cino": "cina", "munafikk": "munafik", "meker": "mikir", "indoneaia": "indonesia", "benarin": "membenarkan", "jaadii": "jadi", "apo": "apa", "nyo": "nya", "miara": "memiara", "jalanani": "menjalani", "sunah2": "sunah-sunah", "sdah": "sudah", "benda2": "benda-benda", "bilang2": "bilang-bilang", "emangnya": "memangnya", "rumpiin": "rumpi", "hotak": "botak", "ikut2": "ikut-ikut", "jawabanny": "jawabannya", "ajj": "saja", "mikir2": "memikir-mikir", "melihara": "memelihara", "oooh": "oh", "hidupx": "hidupnya", "ngefans": "fan", "lsg": "langsung", "uztad": "ustad", "anjingnyaa": "anjingnya", "pegang2": "pegang-pegang", "ilmh": "ilmu", "menasehati": "menasihati", "kelus": "kali", "hubngan": "hubungan", "koclak": "kocak", "bju": "baju", "cinggirnyaaaa": "cinggirnya", "yahhhh": "ya", "miripppp": "mirip", "sedikitlaahh": "sedikitlah", "miriplahhhhhh": "miriplah", "syediih": "sedih", "mirp": "mirip", "baju.a": "bajunya", "wiiiihhhhhhhhh": "wih", "gantheng": "ganteng", "synk": "sayang", "ngfans": "fan", "miriiippp": "mirip", "idungnya": "hidungnya", "wooow": "wow", "panglingg": "pangling", "artisnyaa": "artisnya", "dtggu": "ditunggu", "trnyata": "ternyata", "bgtttttttt": "banget", "rapih": "rapi", "buuun": "bun", "iyaappps": "iya", "sukse": "sukses", "nih2": "nih-nih", "baruu": "baru", "kecptan": "kecepatan", "rmbutnya": "rambutnya", "giti": "begitu", "penganten": "pengantin", "wrn": "warna", "ne": "ini", "cantikkkkk": "cantik", "loksyut": "lokasi syuting", "amfun": "ampun", "k'": "kak", "asuuu": "asu", "kuu": "ku", "dsini": "disini", "dibls": "dibalas", "ceklist": "ceklis", "terusss": "terus", "tlvn": "telepon", "trkrim": "terkirim", "kelaluan": "kelakuan", "guanteng": "ganteng", "cucok": "cocok", "c": "sih", "gueee": "gue", "semangatt": "semangat", "haduh": "aduh", "mlz": "malas", "emak2": "emak-emak", "tante2": "tante-tante", "kepoh": "kepo", "jalanin": "menjalani", "repong": "repot", "ngrusin": "mengurusi", "iy": "iya", "dnk": "dong", "senyumna": "senyumnya", "ngejelek2in": "menjelek-jelekkan", "hotx": "hotnya", "cwnya": "ceweknya", "mbaaahhh": "mbah", "pasangn": "pasangan", "sttus": "status", "aama": "sama", "hadeuuhhhh": "aduh", "lebar2": "lebar-lebar", "sstu": "sesuatu", "serasiii": "serasi", "cocokx": "cocoknya", "emakx": "emaknya", "iniiiiiiiiiiiiiii": "ini", "cocookk": "cocok", "kll": "kalo", "arghhh": "ah", "ungkapin": "ungkapkan", "merrid": "merit", "cocokkkkk": "cocok", "yaolohh": "ya allah", "aee": "saja", "naggung": "menanggung", "battt": "banget", "am": "sama", "tah": "teh", "luthuna": "lucunya", "astagaaa": "astaga", "suaraynyaa": "suaranya", "duanyaa": "duanya", "anjayyyy": "anjay", "coeggg": "coeg", "uaaasssiiiik": "asik", "boss": "bos", "nyubit": "mencubit", "dede": "adek", "gemez": "gemas", "tium": "cium", "gemeeezzzz": "gemas", "ppa": "papa", "papahhhnya": "papahnya", "duuuhh": "duh", "ginting": "ganteng", "bingiiittzz": "banget", "iiihh": "ih", "dedeknya": "adeknya", "hadeeuh": "aduh", "geneeee": "begini", "yooow": "ya", "gemezz": "gemas", "gayax": "gayanya", "infpyunnnn": "ampun", "paraaah": "parah", "papiii": "papi", "kamunyaa": "kamunya", "ngambek": "mengambek", "haduuuh": "aduh", "kapaan": "kapan", "nginjek": "menginjak", "putihnyaa": "putihnya", "ituuuu": "itu", "omaaaaaaakk": "oma", "loncattt": "loncat", "didatengin": "didatangi", "abg": "abang", "kmariiin": "kemarin", "ayoklaa": "ayolah", "jelasssss": "jelas", "kaliiin": "kalian", "ujunggg": "ujung", "kaakaaaa": "kakak", "srruu": "seru", "kannn": "kan", "jmpaaaa": "jumpa", "yukkk": "yuk", "haii": "hai", "syukaaakkk": "suka", "dengeeriin": "dengarkan", "yakaliiiii": "yakali", "atuhh": "atuh", "ngertii": "mengerti", "apalgi": "apalagi", "akuh": "aku", "mesrany": "mesranya", "jendes": "janda", "ngggk": "enggak", "coccokk": "cocok", "msa": "masa", "jendesss": "janda", "bngtttt": "banget", "ponakn": "ponakan", "tantee": "tante", "gadiss": "gadis", "tpy": "tapi", "jnda": "janda", "bngy": "banget", "bujngn": "bujangan", "ngp": "mengapa", "padetan": "padatan", "nyobain": "mencoba", "pcaran": "pacaran", "tdak": "tidak", "prcuma": "percuma", "langgengggg": "langgeng", "sekaliiiii": "sekali", "anehhh": "aneh", "hayal": "khayal", "bedua": "berdua", "bapeerr": "bawa perasaan", "keinget": "teringat", "merekak": "mereka", "kreeenn": "keren", "abisss": "habis", "kocaaaakkk": "kocak", "abiiiss": "habis", "pass": "pas", "personill": "personil", "lengkapp": "lengkap", "ampuuun": "ampun", "iiihhhh": "ih", "flim.a": "filmnya", "trusan.a": "terusannya", "tdung": "tudung", "gangguin": "mengganggu", "lgkah2": "langkah-langkah", "taw": "tau", "cntiiikkk": "cantik", "tegaar": "tegar", "sktlrang": "sekarang", "kangeeeennnnn": "kangen", "tambh": "tambah", "nangiis": "menangis", "nyaaaaaaaaa": "nya", "bundaa": "bunda", "duuhhhhh": "duh", "gaada": "enggak ada", "malsudnya": "maksudnya", "lluuccuu": "lucu", "siappa": "siapa", "martabakk": "martabak", "skali2": "sekali-kali", "kueren2": "keren-keren", "abgnya": "abangnya", "warkop": "warung kopi", "hayuu": "hayo", "kenapaa": "kenapa", "ilang2an": "hilang-hilangan", "ayooook": "ayo", "astagaaaaaaaa": "astaga", "inihhh": "ini", "oom": "om", "mansetan": "bermanset", "cadasss": "cadas", "kereeeeennn": "keren", "temen\"": "teman-teman", "telefisi": "televisi", "nikmatin": "menikmati", "mantappp": "mantap", "mbahh": "mbah", "giitu": "begitu", "mtanya": "matanya", "cpit": "sipit", "dee": "dek", "sumvahh": "sumpah", "tamvann": "tampan", "yap": "ya", "guaaaaaaa": "gua", "capa": "sapa", "ngakakkk": "mengakak", "liaat": "lihat", "gantweng": "ganteng", "anjjjjjjjj": "anjing", "gakwad": "enggak kuat", "seru2": "seru-seru", "iniii": "ini", "favoriteeee": "favorit", "adaa": "ada", "dinyanyiin": "dinyanyikan", "seruu": "seru", "nyengir": "menyengir", "temennya": "temannya", "klk": "kelak", "brjdh": "berjodoh", "akhrt": "akhirat", "kangennnnn": "kangen", "kanggen": "kangen", "kangennnnnnn": "kangen", "ysh": "ya", "sypa": "siapa", "nangisss": "menangis", "brjodoh": "berjodoh", "kaliann": "kalian", "barengg": "bareng", "kpnpun": "kapanpun", "mendj": "menjadi", "allahhhh": "allah", "merindinggg": "merinding", "amiennnn": "amien", "iihh": "ih", "lhir": "lahir", "udeh": "sudah", "debay": "adek bayi", "bangeer": "banget", "lafazd": "lafaz", "pmbyran": "pembayaran", "plsa": "pulsa", "dimana\"": "dimana-mana", "sokk": "sok", "ngambekan": "ambekan", "sma2": "sama-sama", "ngom": "mengomong", "toloollll": "tolol", "bodhooo": "bodoh", "peliharaaaa": "pelihara", "makanyaa": "makanya", "ati\"": "hati-hati", "kemakan": "termakan", "bodhoo": "bodoh", "siihhh": "sih", "andaaa": "anda", "buktinyaaa": "buktinya", "smuaaa": "semua", "ngambeknya": "mengambeknya", "kebongkar": "terbongkar", "tapii": "tapi", "kasiian": "kasihan", "diladenin": "diladeni", "comen": "komen", "ngtif": "negatif", "comntr": "komentar", "kaliannn": "kalian", "brdiri": "berdiri", "mulutny": "mulutnya", "msalah": "masalah", "trserh": "terserah", "waduuuuh": "waduh", "dihpus": "dihapus", "mrekaa": "mereka", "cakeeeeppp": "cakep", "benerr": "benar", "brg\"": "bareng-bareng", "baanget": "banget", "anjaaaayyyyyy": "anjay", "ngetsz": "banget", "utkmu": "untukmu", "anj": "anjing", "merekaa": "mereka", "kann": "kan", "dieee": "dia", "yaahh": "ya", "hijap": "hijab", "ternodain": "ternodai", "ngeri2": "ngeri-ngeri", "mn": "mana", "kt2": "kata-kata", "agk": "agak", "slhkan": "salahkan", "jilbabny": "jilbabnya", "perilakuny": "perilakunya", "ketaatanny": "ketaatannya", "kepda": "kepada", "sblmny": "sebelumnya", "bljr": "belajar", "pedulilaaaah": "pedulilah", "ingetin": "mengingatkan", "terjrumus": "terjerumus", "stuju": "setuju", "igtlah": "ingatlah", "dtgkan": "datangkan", "benahin": "membenahi", "ohhhh": "oh", "yapp": "ya", "jauhh": "jauh", "bingitt": "banget", "buka2an": "buka-bukaan", "ngembali": "kembali", "semngat": "semangat", "lekong": "laki", "kacauuuu": "kacau", "ngimpi": "mimpi", "owh": "oh", "ngerampas": "merampas", "ngehancurin": "menghancurkan", "ngungkit2": "mengungkit-ungkit", "kaleee": "kali", "caranga": "caranya", "usa": "usah", "dipikirin": "dipikirkan", "nunduk": "menunduk", "tembemmm": "tembem", "lucuuuuuuuuuu": "lucu", "bundaaa": "bunda", "nyubitt": "mencubit", "jls2": "jelas-jelas", "supeer": "super", "inni": "ini", "appa": "apa", "hayooo": "hayo", "adyem": "adem", "mitnah": "memfitnah", "cantiknyaaaaa": "cantiknya", "tmbm": "tembam", "cantix": "cantik", "kriput": "keriput", "jelex": "jelek", "drmn": "darimana", "laku\"": "laku-laku", "orang\"nya": "orang-orangnya", "kacian": "kasihan", "cape": "capek", "mancing2": "memancing-mancing", "eeeeh": "eh", "kpancing": "terpancing", "lakuin": "melakukan", "cri": "cari", "sbenernya": "sebenarnya", "mikirinnn": "memikirkan", "diam\"": "diam-diam", "mengalahkn": "mengalahkan", "nyindirin": "menyindir", "nee": "ini", "klakuan": "kelakuan", "mksdx": "maksudnya", "shabat": "sahabat", "brbuat": "berbuat", "mmberi": "memberi", "nm": "nama", "sbhabat": "sahabat", "pnjerumus": "penjerumus", "tman": "teman", "shbat": "sahabat", "melakukn": "melakukan", "berkluarga": "berkeluarga", "ketipu": "tertipu", "laki\"": "laki-laki", "mengait2kan": "mengait-ngaitkan", "nyir2": "nyinyir", "gosipin": "menggosipkan", "sendri": "sendiri", "bntar": "bentar", "duuuhhh": "duh", "nmbk": "menembak", "gakkk": "enggak", "abiiiiisssss": "habis", "tikung2": "tikung-tikung", "cwok": "cowok", "bangedd": "banget", "wuastagah": "astaga", "bedaaa": "beda", "jaoooh": "jauh", "mateeeeeeeeeeee": "mati", "parsis": "persis", "sumpeh": "sumpah", "iyh": "iya", "kmana": "kemana", "cantiqs": "cantik", "kecik": "kecil", "brasa": "berasa", "trlihat": "terlihat", "mmberikan": "memberikan", "khidupan": "kehidupan", "srpti": "seperti", "dinamakn": "dinamakan", "sgtu": "sebegitu", "paati": "pasti", "kutuu": "kutu", "setelur2nya": "setelur-telurnya", "khadiran": "kehadiran", "srta": "serta", "kcil": "kecil", "kberkahn": "keberkahan", "kbahagiaan": "kebahagiaan", "tntram": "tentram", "brsama": "bersama", "hngga": "hingga", "brpndapat": "berpendapat", "trutama": "terutama", "hiraukn": "hiraukan", "diamkn": "diamkan", "hrsny": "harusny", "brsyukur": "bersyukur", "mmberikn": "memberikan", "rasakn": "rasakan", "cari2": "cari-cari", "kesalahanny": "kesalahannya", "kekuranganny": "kekurangannya", "seandainy": "seandainya", "perasaanny": "perasaannya", "diindonesis": "diindonesia", "liyatnya": "lihatnya", "kayahhh": "kaya", "ala2": "ala-ala", "yook": "yuk", "yoookkkk": "yuk", "siihh": "sih", "elus2": "elus-elus", "asliiiii": "asli", "aduuhhhh": "aduh", "padahl": "padahal", "btul": "betul", "kejahatn": "kejahatan", "slam": "salam", "ngelewati": "melewati", "kalu": "kalau", "ngerendahin": "merendahkan", "apaalahh": "apalah", "menyebut2": "menyebut-nyebut", "kiya": "kita", "miriiiiip": "mirip", "dirimuuu": "dirimu", "yoiiu": "yoi", "semuanyaaaa": "semuanya", "suka.a": "sukanya", "jdii": "jadi", "blnja": "belanja", "brangnya": "barangnya", "nimpuk": "menimpuk", "abiss": "habis", "prlu": "perlu", "coyyy": "coy", "suaranyaa": "suaranya", "jnagn": "jangan", "kcwa": "kecewa", "jaman2": "jaman-jaman", "dket2in": "dekat-dekati", "cobak": "coba", "britanya": "beritanya", "mahh": "mah", "jamam": "jaman", "paraa": "para", "seharusny": "seharusnya", "dasaar": "dasar", "bergunaa": "berguna", "mulutnyaaa": "mulutnya", "blkng": "belakang", "kcewa": "kecewa", "mnusuk": "menusuk", "blkang": "belakang", "nembak": "menembak", "trmanis": "termanis", "kaciiaann": "kasihan", "nyela": "mencela", "ngejilat": "menjilat", "hallooo": "halo", "deketin": "dekati", "tayik": "tahi", "halooo": "halo", "ustzah": "ustazah", "halow": "halo", "poooolllll": "pol", "tahannn": "tahan", "aduuh": "aduh", "nyimpen": "menyimpan", "ngarsipin": "mengarsipkan", "cman": "cuman", "gituuuuu": "begitu", "kyy": "kaya", "kngn": "kangen", "kq": "kok", "nyesek": "menyesak", "rinduuuuuuu": "rindu", "kangeeeennn": "kangen", "mg": "moga", "aamiiiiinnn": "amin", "gemezzzz": "gemas", "kangennnnnn": "kangen", "prh": "parah", "bakattt": "bakat", "napsuu": "napsu", "gmau": "enggak mau", "luccu": "lucu", "prasaan": "perasaan", "lahirr": "lahir", "setujuuuu": "setuju", "ceemen": "cemen", "sembunyi\"\"": "sembunyi-sembunyi", "skrnh": "sekarang", "cerdiik": "cerdik", "jeniuss": "jenius", "kurangin": "kurangi", "kreen": "keren", "s'lalu": "selalu", "mauuuuuu": "mau", "mintk": "minta", "nyolong": "mencolong", "pnjng": "panjang", "slaluuuuu": "selalu", "kanggggg": "kang", "rezeqinya": "rezekinya", "pintr": "pintar", "dekk": "dek", "maw": "mau", "kura2": "kura-kura", "edaaann": "edan", "nempel": "menempel", "hatiiii": "hati", "sukkaa": "suka", "luuuucuuu": "lucu", "caantiiik": "cantik", "luucuuu": "lucu", "putihh": "putih", "terus\"an": "terus-terusan", "siiih": "sih", "ternyataa": "ternyata", "pollll": "pol", "sekaliiii": "sekali", "peduliin": "mempedulikan", "komntr": "komentar", "smngat": "semangat", "dosax": "dosanya", "pahalax": "pahalanya", "akhirt": "akhirat", "tinggalkn": "tinggalkan", "rumh": "rumah", "mk": "maka", "trhtung": "terhitung", "naaaa": "nya", "ademmm": "adem", "cerdass": "cerdas", "tambahh": "tambah", "polll": "pol", "didengerin": "didengarkan", "mantapppppp": "mantap", "ngerespon": "merespons", "dulurrr": "dulur", "brusaha": "berusaha", "lebh": "lebih", "amin2": "amin-amin", "positippp": "positif", "jauuhhh": "jauh", "brhjb": "berhijab", "mdh2n": "mudah-mudahan", "slmnya": "salamnya", "jlbb": "jilbab", "dempga": "semoga", "selmnya": "selamanya", "tth": "teteh", "otot2": "otot-otot", "lengkeeett": "lengket", "caaannnttiikkk": "cantik", "trusss": "terus", "ademm": "adem", "apalagiii": "apalagi", "passs": "pas", "beginii": "begini", "satuu": "satu", "cuantek": "cantik", "akuuuur": "akur", "iniehh": "ini", "sayangx": "sayangnya", "jodohx": "jodohnya", "kaliaan": "kalian", "haslnya": "hasilnya", "kencng": "kencang", "pmbantu": "pembantu", "kmbl": "kembali", "cuchook": "cocok", "kloop": "klop", "sempurnaa": "sempurna", "klihatan": "kelihatan", "cintax": "cintanya", "pnuh": "penuh", "ngrti": "mengerti", "mudah\"an": "mudah-mudahan", "segala\"ny": "segala-galanya", "mantanny": "mantannya", "jodohny": "jodohnya", "cantk": "cantik", "ituuu": "itu", "kaasian": "kasihan", "padhl": "padahal", "ank\"": "anak-anak", "pkrin": "pikirkan", "anakx": "anaknya", "sia2kan": "sia-siakan", "naaahhh": "nah", "beluumm": "belum", "sayangggq": "sayangku", "kreean": "keren", "ciiieee": "cie", "cieee": "cie", "cieeee": "cie", "berenanh": "berenang", "pacarr": "pacar", "bingitss": "banget", "besokkk": "besok", "annnnjjjjjjrrrr": "anjir", "ngeliatin": "melihati", "ngumbar2": "mengumbar-umbar", "romantiss": "romantis", "cocook": "cocok", "lucuuuuuuuu": "lucu", "kaaaaaann": "kan", "nyaaaaaaa": "nya", "lgs": "langsung", "yaang": "yang", "bangedt": "banget", "gde": "gede", "lutunaaa": "lucunya", "bikinnn": "bikin", "keee": "ke", "bungkuuussss": "bungkus", "lucux": "lucunya", "buanyaaak": "banyak", "aaamiiinnn": "amin", "aaamiiin": "amin", "abgnyaah": "abangnya", "mungkiin": "mungkin", "akuhh": "aku", "manteb": "mantap", "hiduupppp": "hidup", "tebel": "tebal", "nganggepin": "menanggapi", "mentang\"": "mentang-mentang", "najiss": "najis", "perjabatt": "pejabat", "makany": "makanya", "benneeeeeerr": "benar", "krjaan": "pekerjaan", "kdang": "kadang", "knan": "kanan", "anak'y": "anaknya", "trknal": "terkenal", "gtu\"": "begitu-gitu", "klok": "kalo", "aamiiiinnnnn": "amin", "pengennnn": "pengin", "guw": "gue", "siiiiiiippppppppp": "sip", "selaloe": "selalu", "donggg": "dong", "beruntungnyaaaa": "beruntungnya", "soal'y": "soalnya", "prhatian": "perhatian", "hebaat": "hebat", "ampunn": "ampun", "gueeeee": "gue", "liatt": "lihat", "gehh": "gih", "ankknyaaa": "anaknya", "centilnyaaaaaa": "centilnya", "gemesshzzzzhzzhzhzhsjssjs": "gemas", "ngacaaa": "mengaca", "diaaa": "dia", "jirr": "anjir", "kayaa": "kaya", "cempreeng": "cempreng", "centillll": "centil", "pcran": "pacaran", "anedh": "banget", "ampyunnnn": "ampun", "boro2": "boro-boro", "nyta": "nyata", "kampungn": "kampungan", "j": "saja", "mnghina": "menghina", "ngolokin": "mengolok", "blum": "belum", "syirik\"": "syirik-syirik", "than": "tahan", "siriik": "sirik", "mampuuu": "mampu", "khan": "kan", "aduuhh": "aduh", "cemburuu": "cemburu", "bingitzzzz": "banget", "tlog": "tolong", "apasihhh": "apasih", "sirikk": "sirik", "dbnding": "dibanding", "nrka": "neraka", "s3ndiri": "sendiri", "bhgiain": "membahagiakan", "kdua": "kedua", "prcya": "percaya", "bhagaia": "bahagia", "cuco": "cocok", "genittt": "genit", "kx": "kak", "mlikk": "milik", "mentah\"": "mentah-mentah", "canci": "cantik", "aneed": "banget", "gimanaa": "bagaimana", "lainnn": "lain", "pentingg": "penting", "bagaimanaaa": "bagaimana", "krudung": "kerudung", "enakkkkk": "enak", "poool": "pol", "jirrrr": "anjir", "oprasiya": "operasinya", "uwee": "gue", "pilihannny": "pilihannya", "itumahh": "itumah", "muuuu": "mu", "uuhhh": "uh", "berbakiti": "berbakti", "alhamd": "alhamdulillah", "slmttt": "selamat", "ucapin": "mengucapkan", "ditengah2": "ditengah-tengah", "semuanyaa": "semuanya", "semuannyaaa": "semuanya", "aminn2": "amin-amin", "abs": "habis", "kebnyak'an": "kebanyakan", "melahirin": "melahirkan", "aduuuh": "aduh", "cakeep": "cakep", "lhooo": "loh", "kamuuuu": "kamu", "bagi2": "bagi-bagi", "gimn": "bagaimana", "smogaaa": "semoga", "biza": "bisa", "aamien": "amien", "lngsing": "langsing", "wowoooww": "wow", "smulla": "semula", "pdhall": "padahal", "blln": "bulan", "senang2": "senang-senang", "jh": "saja", "kereeennnn": "keren", "gilakkk": "gila", "gtu2": "begitu-gitu", "cpet": "cepat", "gataunya": "enggak taunya", "gtau": "enggak tau", "fans2": "fan-fan", "gaharus": "enggak harus", "&": "dan"}
print(slangs_1)

Slang words dari https://github.com/nasalsabila/kamus-alay/blob/master/colloquial-indonesian-lexicon.csv

In [ ]:
data_slangs = pd.read_csv('./datasets/slangs/slangs_1.csv')
slangs_2 = data_slangs[['slang', 'formal']].set_index('slang').to_dict()['formal']
print(slangs_2)

Slang word tambahan manual

In [ ]:
slangs_3 = {
    'yg': 'yang',
    'cantek': 'cantik',
    'hepibesdey': 'selamat ulang tahun',
    'ga' : 'tidak',
    'enggak' : 'tidak',
    'engak' : 'tidak',
    'nyante' : 'santai',
    'sruput' : 'minum',
    'gue' : 'saya',
    'lgsng' : 'langsung',
    'kalo' : 'kalau',
    'laper' : 'lapar',
    'nolak' : 'tolak',
    'anyversary' : 'aniversary',
    'pdli' : 'peduli',
    'trsrah' : 'terserah',
    'rasai' : 'rasain',
    'slma' : 'selama',
    'nga' : 'tidak'
}

Gabungkan dictionary slang yang ada

In [ ]:
all_slangs = {**slangs_1, **slangs_2, **slangs_3}
all_slangs = dict(sorted(all_slangs.items()))

print(len(all_slangs))

In [ ]:
all_slangs

Check slangs, kalau ga ada tambahin manual di slangs_3

In [ ]:
# Ubah ini
slang_to_check = 'engak'

if slang_to_check in all_slangs:
    print(slang_to_check + " : " + all_slangs[slang_to_check])
else:
    print('Slang word belum ada, tambahkan manual di slang_3')

Hapus karakter berulang di slangword

In [ ]:
new_slangs = {}
for key, value in all_slangs.items():
    new_key = clean_repeating_characters(key)
    if new_key not in new_slangs:
        new_value = clean_repeating_characters(value)
        result = re.search(r"\b" + re.escape(new_key) + r"\b", new_value)
        if new_key != new_value and result == None:
            new_slangs[new_key] = new_value

In [ ]:
def clean_puctuations_slangs(text):
    translator = str.maketrans('-"', ' ' * len('-"'))
    text = text.translate(translator)
    return text

In [ ]:
processed_slangs = {}
for key, value in new_slangs.items():
    new_key = clean_puctuations_slangs(key)
    new_key = re.sub(' ', '', new_key)
    if new_key.lower() not in processed_slangs:
        new_value = clean_puctuations_slangs(value)
        result = re.search(r"\b" + re.escape(new_key) + r"\b", new_value)
        if new_key != new_value and result == None:
            processed_slangs[new_key.lower()] = new_value.lower()

In [ ]:
# Ubah ini
slang_to_check = 'hati'

if slang_to_check in processed_slangs:
    print(slang_to_check + " : " + processed_slangs[slang_to_check])
else:
    print('Slang word belum ada, tambahkan manual di slang_3')

In [ ]:
processed_slangs

In [ ]:
print(len(processed_slangs))

Hapuskan slangword

In [ ]:
# def remove_slangs(text):
#     for word in processed_slangs.keys():
#         pattern = r'\b' + re.escape(word) + r'\b'
#         text = re.sub(pattern, processed_slangs[word], text)
#     return text

In [ ]:
def remove_slangs(text):
    # Create a pattern to match all slangs using alternation (|) and word boundaries (\b)
    pattern = r'\b(?:' + '|'.join(map(re.escape, processed_slangs.keys())) + r')\b'
    
    # Replace slangs with their dictionary values
    return re.sub(pattern, lambda x: processed_slangs.get(x.group(0), x.group(0)), text)

In [ ]:
n = 2000
while n > 1008:
    initial_df = train.copy()
    train["tweet"] = train["tweet"].apply(remove_slangs)
    n = count_changes(initial_df, train)
    print("Banyak perubahan pada step ini = " , n)

In [ ]:
# train

### Stemming

In [ ]:
!pip install Sastrawi

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Create a Sastrawi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
def stem_text(text):
    return stemmer.stem(text)

In [ ]:
initial_df = train.copy()
train["tweet"] = train["tweet"].apply(stem_text)
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

In [ ]:
# train

### Remove stopwords 

In [ ]:
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('indonesian')) # ganti 'indonesian' ke bahasa lain kalau dibutuhin

Tambahkan stopword manual disini. Stopword disini didapat dari wordcloud. 

In [ ]:
additional_stopwords = {
    "manusia",
    "banget",
    "sih",
    "ya",
    'sat'
    }

In [ ]:
stop_words.update(additional_stopwords)
STOPWORDS = set(stop_words)

Hapus stopword yang seharusnya dibutuhkan

In [ ]:
# remove_stopword = {
#     'baik'
# }

In [ ]:
# for word in remove_stopword:
#     STOPWORDS.discard(word)

In [ ]:
STOPWORDS

Check apakah suatu kata ada pada stopword atau tidak

In [ ]:
# Ubah ini
stopword_to_check = 'manusia'

if stopword_to_check in STOPWORDS:
    print(stopword_to_check + ' sudah ada')
else:
    print('Stopword belum ada, tambahkan manual di additional_stopwords')

Hapuskan stopword

In [ ]:
def clean_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [ ]:
initial_df = train.copy()
train.loc[:, 'tweet'] = train['tweet'].apply(lambda text: clean_stopwords(text))
print("Banyak perubahan pada step ini = " , count_changes(initial_df, train))

In [ ]:
# train

### Hasil Akhir Preprocessing Tweet 

In [ ]:
train

In [ ]:
print(train.isnull().sum())

In [ ]:
train.to_csv('./datasets/preprocessed/data.csv', index=False)

### Split Train dan Test

In [ ]:
train = pd.read_csv('./datasets/preprocessed/data.csv')

In [ ]:
train_data = train.iloc[:-1000]
train_data.to_csv('./datasets/preprocessed/train.csv', index=False)

test_data = train.iloc[-1000:]
test_data.to_csv('./datasets/preprocessed/test.csv', index=False)

### Remove Duplicate Data Train

In [ ]:
print(train_data.isnull().sum())

In [ ]:
duplicates = train_data[train_data.duplicated()]

print(len(duplicates))

In [ ]:
first_duplicate = duplicates.iloc[0]  # Get the first duplicate row

matching_rows = train_data[train_data['tweet'] == first_duplicate['tweet']]

matching_rows

Remove Duplicates

In [ ]:
train_data = train_data.drop_duplicates()

In [ ]:
duplicates = train_data[train_data.duplicated()]

print(len(duplicates))

Sudah tidak ada data duplicates

In [ ]:
train_data.dropna(subset=['tweet'], inplace=True)
print(train_data.isnull().sum())

In [ ]:
train_data.to_csv('./datasets/no_duplicate/train.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c0407355-6900-4af1-8f60-c3569a9b303d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>